# 🧠 EGM-Net: Energy-Gated Gabor Mamba Network

**Medical Image Segmentation with Implicit Neural Representations**

## 1️⃣ Setup

In [ ]:
# Install dependencies
%pip install mamba-ssm -q torch torchvision numpy matplotlib tqdm gdown nibabel scikit-image monai


## 2️⃣ Model Architecture (Standalone SOTA Implementation)


Full implementation of EGM-Net, HRNetV2-Mamba, and Physics-based Layers.


### data_utils.py


In [ ]:
"""
Data utilities for medical image segmentation with Monogenic Signal support.

Includes:
- JointVectorRotation: Augmentation that rotates images and Riesz vectors synchronously
- MonogenicDataset: Dataset with pre-computed Monogenic Signal components
- MedicalImageSegmentationDataset: Base dataset class
- MetricsCalculator: Segmentation metrics computation
"""

import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import numpy as np
import math
import random
from pathlib import Path
from torch.utils.data import Dataset
from typing import Tuple, Optional, Dict, List, Union


# =============================================================================
# Augmentation Transforms
# =============================================================================

class JointVectorRotation:
    """
    Synchronously rotate image, Riesz vectors, and mask.
    
    When rotating an image, the Riesz vector field (Rx, Ry) must be:
    1. Geometrically rotated (pixel positions move)
    2. Vector-rotated (direction at each pixel changes)
    
    This avoids expensive FFT re-computation during augmentation.
    
    Mathematical basis:
        If image rotates by angle θ, then at each pixel:
        [Rx']   [cos(θ)  -sin(θ)] [Rx]
        [Ry'] = [sin(θ)   cos(θ)] [Ry]
    
    Args:
        angle_range: Tuple of (min_angle, max_angle) in degrees
        p: Probability of applying rotation
    """
    
    def __init__(self, angle_range: Tuple[float, float] = (-180, 180), p: float = 1.0):
        self.angle_range = angle_range
        self.p = p
    
    def __call__(self, image: torch.Tensor, riesz_vec: torch.Tensor, 
                 mask: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Apply synchronized rotation.
        
        Args:
            image: Intensity image (1, H, W) or (C, H, W)
            riesz_vec: Riesz components (2, H, W) containing [Rx, Ry]
            mask: Segmentation mask (H, W) or (1, H, W)
            
        Returns:
            Tuple of (rotated_image, rotated_riesz, rotated_mask)
        """
        if random.random() > self.p:
            return image, riesz_vec, mask
        
        angle = random.uniform(self.angle_range[0], self.angle_range[1])
        
        # 1. Geometric Rotation (pixel positions)
        image = TF.rotate(image, angle, interpolation=TF.InterpolationMode.BILINEAR)
        mask = TF.rotate(mask.unsqueeze(0) if mask.dim() == 2 else mask, 
                        angle, interpolation=TF.InterpolationMode.NEAREST)
        if mask.dim() == 3 and mask.shape[0] == 1:
            mask = mask.squeeze(0)
        riesz_vec = TF.rotate(riesz_vec, angle, interpolation=TF.InterpolationMode.BILINEAR)
        
        # 2. Vector Re-orientation (direction at each pixel)
        # Negative angle because image coordinate y-axis is inverted
        theta = torch.tensor(-angle * math.pi / 180.0, dtype=riesz_vec.dtype)
        cos_a = torch.cos(theta)
        sin_a = torch.sin(theta)
        
        rx, ry = riesz_vec[0], riesz_vec[1]
        
        # 2D rotation matrix applied element-wise
        new_rx = rx * cos_a - ry * sin_a
        new_ry = rx * sin_a + ry * cos_a
        
        new_riesz = torch.stack([new_rx, new_ry], dim=0)
        
        return image, new_riesz, mask


class JointRandomFlip:
    """
    Synchronously flip image, Riesz vectors, and mask.
    
    When flipping, Riesz vector components must also be negated appropriately:
    - Horizontal flip: Rx -> -Rx
    - Vertical flip: Ry -> -Ry
    
    Args:
        p_horizontal: Probability of horizontal flip
        p_vertical: Probability of vertical flip
    """
    
    def __init__(self, p_horizontal: float = 0.5, p_vertical: float = 0.5):
        self.p_horizontal = p_horizontal
        self.p_vertical = p_vertical
    
    def __call__(self, image: torch.Tensor, riesz_vec: torch.Tensor,
                 mask: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Apply synchronized flip."""
        
        # Horizontal flip
        if random.random() < self.p_horizontal:
            image = TF.hflip(image)
            mask = TF.hflip(mask.unsqueeze(0) if mask.dim() == 2 else mask)
            if mask.dim() == 3 and mask.shape[0] == 1:
                mask = mask.squeeze(0)
            riesz_vec = TF.hflip(riesz_vec)
            # Negate Rx component
            riesz_vec[0] = -riesz_vec[0]
        
        # Vertical flip
        if random.random() < self.p_vertical:
            image = TF.vflip(image)
            mask = TF.vflip(mask.unsqueeze(0) if mask.dim() == 2 else mask)
            if mask.dim() == 3 and mask.shape[0] == 1:
                mask = mask.squeeze(0)
            riesz_vec = TF.vflip(riesz_vec)
            # Negate Ry component
            riesz_vec[1] = -riesz_vec[1]
        
        return image, riesz_vec, mask


# =============================================================================
# Dataset Classes
# =============================================================================

class MonogenicDataset(Dataset):
    """
    Dataset with pre-computed Monogenic Signal components.
    
    Expects data in .npy format with structure:
    - intensity: (H, W) grayscale image
    - riesz_x: (H, W) Riesz x-component
    - riesz_y: (H, W) Riesz y-component
    - mask: (H, W) segmentation mask
    
    The Riesz components should be pre-computed offline to avoid FFT overhead.
    
    Args:
        data_dir: Directory containing .npy files
        img_size: Target image size for resizing
        augment: Whether to apply data augmentation
        normalize: Whether to normalize intensity to [0, 1]
    """
    
    def __init__(self, data_dir: Union[str, Path], img_size: int = 256,
                 augment: bool = True, normalize: bool = True):
        self.data_dir = Path(data_dir)
        self.img_size = img_size
        self.augment = augment
        self.normalize = normalize
        
        # Find all sample files
        self.samples = sorted(list(self.data_dir.glob("*.npy")))
        
        if len(self.samples) == 0:
            raise ValueError(f"No .npy files found in {data_dir}")
        
        # Augmentation transforms
        if augment:
            self.rotation = JointVectorRotation(angle_range=(-30, 30), p=0.5)
            self.flip = JointRandomFlip(p_horizontal=0.5, p_vertical=0.5)
        else:
            self.rotation = None
            self.flip = None
    
    def __len__(self) -> int:
        return len(self.samples)
    
    def _load_sample(self, path: Path) -> Dict[str, np.ndarray]:
        """Load a sample from .npy file."""
        data = np.load(path, allow_pickle=True).item()
        return data
    
    def _preprocess(self, data: Dict[str, np.ndarray]) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Preprocess loaded data."""
        # Extract components
        intensity = torch.from_numpy(data['intensity']).float()
        riesz_x = torch.from_numpy(data['riesz_x']).float()
        riesz_y = torch.from_numpy(data['riesz_y']).float()
        mask = torch.from_numpy(data['mask']).long()
        
        # Add channel dimension to intensity if needed
        if intensity.dim() == 2:
            intensity = intensity.unsqueeze(0)  # (1, H, W)
        
        # Stack Riesz components
        riesz_vec = torch.stack([riesz_x, riesz_y], dim=0)  # (2, H, W)
        
        # Normalize intensity
        if self.normalize:
            i_min, i_max = intensity.min(), intensity.max()
            if i_max > i_min:
                intensity = (intensity - i_min) / (i_max - i_min)
        
        return intensity, riesz_vec, mask
    
    def _resize(self, intensity: torch.Tensor, riesz_vec: torch.Tensor, 
                mask: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Resize to target size."""
        if intensity.shape[-1] != self.img_size or intensity.shape[-2] != self.img_size:
            intensity = F.interpolate(
                intensity.unsqueeze(0), size=(self.img_size, self.img_size),
                mode='bilinear', align_corners=True
            ).squeeze(0)
            
            riesz_vec = F.interpolate(
                riesz_vec.unsqueeze(0), size=(self.img_size, self.img_size),
                mode='bilinear', align_corners=True
            ).squeeze(0)
            
            mask = F.interpolate(
                mask.float().unsqueeze(0).unsqueeze(0),
                size=(self.img_size, self.img_size),
                mode='nearest'
            ).squeeze(0).squeeze(0).long()
        
        return intensity, riesz_vec, mask
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        """
        Get a sample.
        
        Returns:
            Dictionary with:
            - 'input': Combined input (3, H, W) = [Intensity, Rx, Ry]
            - 'intensity': (1, H, W)
            - 'riesz': (2, H, W)
            - 'mask': (H, W)
            - 'energy': (1, H, W) computed from Riesz components
        """
        # Load and preprocess
        data = self._load_sample(self.samples[idx])
        intensity, riesz_vec, mask = self._preprocess(data)
        
        # Resize
        intensity, riesz_vec, mask = self._resize(intensity, riesz_vec, mask)
        
        # Augmentation (AFTER loading, with vector rotation)
        if self.augment:
            if self.rotation is not None:
                intensity, riesz_vec, mask = self.rotation(intensity, riesz_vec, mask)
            if self.flip is not None:
                intensity, riesz_vec, mask = self.flip(intensity, riesz_vec, mask)
        
        # Compute energy from monogenic components
        # E = sqrt(I^2 + Rx^2 + Ry^2)
        energy = torch.sqrt(
            intensity ** 2 + riesz_vec[0:1] ** 2 + riesz_vec[1:2] ** 2 + 1e-8
        )
        # Normalize energy to [0, 1]
        energy = (energy - energy.min()) / (energy.max() - energy.min() + 1e-8)
        
        # Combine into 3-channel input
        combined_input = torch.cat([intensity, riesz_vec], dim=0)  # (3, H, W)
        
        return {
            'input': combined_input,
            'intensity': intensity,
            'riesz': riesz_vec,
            'mask': mask,
            'energy': energy
        }





class MedicalImageSegmentationDataset(Dataset):
    """
    Base dataset class for medical image segmentation.
    Can be extended for specific data formats (NIfTI, DICOM, PNG, etc.)
    """
    
    def __init__(self, images: np.ndarray, masks: np.ndarray,
                 img_size: int = 256, normalize: bool = True,
                 augment: bool = False):
        """
        Initialize dataset.
        
        Args:
            images: Array of shape (N, H, W) or (N, C, H, W)
            masks: Array of shape (N, H, W) with class labels
            img_size: Target image size for resizing
            normalize: Whether to normalize images (0-1 or standardize)
            augment: Whether to apply data augmentation
        """
        self.images = images
        self.masks = masks
        self.img_size = img_size
        self.normalize = normalize
        self.augment = augment
        
        assert len(images) == len(masks), "Images and masks must have same length"
        
        # Ensure 4D shape (N, C, H, W)
        if self.images.ndim == 3:
            self.images = np.expand_dims(self.images, axis=1)
        
        # Preprocess
        self.images = torch.from_numpy(self.images).float()
        self.masks = torch.from_numpy(self.masks).long()
        
        if self.normalize:
            self._normalize_images()
    
    def _normalize_images(self):
        """Normalize images to 0-1 range or standardize."""
        # Normalize per image to 0-1
        for i in range(len(self.images)):
            img = self.images[i]
            img_min = img.min()
            img_max = img.max()
            if img_max > img_min:
                self.images[i] = (img - img_min) / (img_max - img_min)
    
    def _resize_if_needed(self, image: torch.Tensor, mask: torch.Tensor) -> Tuple:
        """Resize image and mask if needed."""
        if image.shape[-1] != self.img_size or image.shape[-2] != self.img_size:
            image = F.interpolate(
                image.unsqueeze(0), size=(self.img_size, self.img_size),
                mode='bilinear', align_corners=True
            ).squeeze(0)
            mask = F.interpolate(
                mask.float().unsqueeze(0).unsqueeze(0),
                size=(self.img_size, self.img_size),
                mode='nearest'
            ).squeeze(0).squeeze(0).long()
        return image, mask
    
    def _augment_data(self, image: torch.Tensor, mask: torch.Tensor) -> Tuple:
        """Apply basic data augmentation."""
        # Random horizontal flip
        if torch.rand(1).item() > 0.5:
            image = torch.flip(image, dims=[-1])
            mask = torch.flip(mask, dims=[-1])
        
        # Random vertical flip
        if torch.rand(1).item() > 0.5:
            image = torch.flip(image, dims=[-2])
            mask = torch.flip(mask, dims=[-2])
        
        # Random rotation (0, 90, 180, 270)
        k = torch.randint(0, 4, (1,)).item()
        image = torch.rot90(image, k=k, dims=[-2, -1])
        mask = torch.rot90(mask, k=k, dims=[-2, -1])
        
        return image, mask
    
    def __len__(self) -> int:
        return len(self.images)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        image = self.images[idx]
        mask = self.masks[idx]
        
        # Resize if needed
        image, mask = self._resize_if_needed(image, mask)
        
        # Augmentation
        if self.augment:
            image, mask = self._augment_data(image, mask)
        
        return image, mask


class MetricsCalculator:
    """Calculate segmentation metrics."""
    
    @staticmethod
    def dice_score(pred: torch.Tensor, target: torch.Tensor, 
                   smooth: float = 1e-5) -> float:
        """
        Calculate Dice coefficient.
        
        Args:
            pred: Predictions of shape (B, C, H, W) or (B, H, W)
            target: Ground truth of shape (B, H, W)
            smooth: Smoothing constant
            
        Returns:
            Dice score (0-1)
        """
        if pred.ndim == 4:
            pred = torch.argmax(pred, dim=1)
        
        pred = pred.view(-1)
        target = target.view(-1)
        
        intersection = (pred == target).sum().float()
        union = pred.numel()
        
        dice = (2.0 * intersection + smooth) / (union + smooth)
        return dice.item()
    
    @staticmethod
    def iou_score(pred: torch.Tensor, target: torch.Tensor,
                  num_classes: int = 3, smooth: float = 1e-5) -> dict:
        """
        Calculate Intersection over Union (IoU) for each class.
        
        Args:
            pred: Predictions of shape (B, C, H, W) or (B, H, W)
            target: Ground truth of shape (B, H, W)
            num_classes: Number of classes
            smooth: Smoothing constant
            
        Returns:
            Dictionary with per-class and mean IoU
        """
        if pred.ndim == 4:
            pred = torch.argmax(pred, dim=1)
        
        iou_scores = {}
        mean_iou = 0.0
        
        for cls in range(num_classes):
            pred_mask = (pred == cls)
            target_mask = (target == cls)
            
            intersection = (pred_mask & target_mask).sum().float()
            union = (pred_mask | target_mask).sum().float()
            
            iou = (intersection + smooth) / (union + smooth)
            iou_scores[f"class_{cls}"] = iou.item()
            mean_iou += iou.item()
        
        iou_scores["mean"] = mean_iou / num_classes
        return iou_scores
    
    @staticmethod
    def hausdorff_distance(pred: torch.Tensor, target: torch.Tensor) -> float:
        """
        Calculate Hausdorff distance (boundary metric).
        
        Args:
            pred: Predictions of shape (B, H, W)
            target: Ground truth of shape (B, H, W)
            
        Returns:
            Hausdorff distance
        """
        # Simple implementation using max of minimum distances
        pred = pred.float().view(-1, 1)
        target = target.float().view(-1, 1)
        
        # Distance from pred to target
        dist_pt = torch.cdist(pred, target).min(dim=1)[0]
        max_dist_pt = dist_pt.max()
        
        # Distance from target to pred
        dist_tp = torch.cdist(target, pred).min(dim=1)[0]
        max_dist_tp = dist_tp.max()
        
        hd = max(max_dist_pt.item(), max_dist_tp.item())
        return hd
    
    @staticmethod
    def sensitivity_specificity(pred: torch.Tensor, target: torch.Tensor,
                               num_classes: int = 2) -> dict:
        """
        Calculate sensitivity and specificity for binary/multi-class.
        
        Args:
            pred: Predictions of shape (B, C, H, W) or (B, H, W)
            target: Ground truth of shape (B, H, W)
            num_classes: Number of classes
            
        Returns:
            Dictionary with sensitivity and specificity per class
        """
        if pred.ndim == 4:
            pred = torch.argmax(pred, dim=1)
        
        metrics = {}
        
        for cls in range(1, num_classes):  # Skip background class 0
            pred_pos = (pred == cls)
            pred_neg = (pred != cls)
            target_pos = (target == cls)
            target_neg = (target != cls)
            
            tp = (pred_pos & target_pos).sum().float().item()
            tn = (pred_neg & target_neg).sum().float().item()
            fp = (pred_pos & target_neg).sum().float().item()
            fn = (pred_neg & target_pos).sum().float().item()
            
            sensitivity = tp / (tp + fn + 1e-5)
            specificity = tn / (tn + fp + 1e-5)
            
            metrics[f"class_{cls}"] = {
                "sensitivity": sensitivity,
                "specificity": specificity
            }
        
        return metrics




### spectral_layers.py


In [ ]:
"""
Spectral Layers Module
Implements frequency domain processing using FFT-based gating.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional


class SpectralGating(nn.Module):
    """
    Spectral Gating Module using FFT for frequency domain filtering.
    
    Applies learnable frequency filtering in the frequency domain to enhance
    edge sharpness and remove noise by modulating frequency components.
    """
    
    def __init__(self, channels: int, height: int, width: int, 
                 threshold: float = 0.1, complex_init: str = "kaiming"):
        """
        Initialize SpectralGating module.
        
        Args:
            channels: Number of input channels
            height: Input height (should be divisible by some factor)
            width: Input width (should be divisible by some factor)
            threshold: Hard thresholding value for amplitude (0 to disable)
            complex_init: Initialization strategy ("kaiming" or "identity")
        """
        super().__init__()
        self.channels = channels
        self.height = height
        self.width = width
        self.threshold = threshold
        
        # Create learnable complex weights for frequency domain
        # Shape: (channels, height, width//2 + 1) for rfft2
        self.register_buffer(
            "freq_shape",
            torch.tensor([channels, height, width // 2 + 1], dtype=torch.long)
        )
        
        # Real and Imaginary parts of complex weights
        self.weight_real = nn.Parameter(
            torch.zeros(channels, height, width // 2 + 1)
        )
        self.weight_imag = nn.Parameter(
            torch.zeros(channels, height, width // 2 + 1)
        )
        
        self._init_weights(complex_init)
        
    def _init_weights(self, strategy: str = "kaiming"):
        """Initialize complex weights."""
        if strategy == "identity":
            # Initialize close to identity (magnitude ~1, phase ~0)
            nn.init.ones_(self.weight_real)
            nn.init.zeros_(self.weight_imag)
        elif strategy == "kaiming":
            # Kaiming initialization adapted for complex numbers
            fan_in = self.height * (self.width // 2 + 1)
            std = (2.0 / fan_in) ** 0.5
            nn.init.normal_(self.weight_real, 0, std)
            nn.init.normal_(self.weight_imag, 0, std)
        else:
            raise ValueError(f"Unknown init strategy: {strategy}")
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Apply spectral gating to input.
        
        Args:
            x: Input tensor of shape (B, C, H, W)
            
        Returns:
            Filtered output tensor of shape (B, C, H, W)
        """
        B, C, H, W = x.shape
        
        # Apply FFT to convert to frequency domain
        # rfft2 returns complex tensor
        x_freq = torch.fft.rfft2(x, dim=(-2, -1), norm="ortho")
        
        # Create complex weight matrix: weight_real + 1j * weight_imag
        # Reshape to (1, C, H, W//2+1) for broadcasting
        complex_weight = (
            self.weight_real.unsqueeze(0) + 
            1j * self.weight_imag.unsqueeze(0)
        )
        
        # Apply channel-wise multiplication in frequency domain
        # Shape: (B, C, H, W//2+1) * (1, C, H, W//2+1) -> (B, C, H, W//2+1)
        x_filtered = x_freq * complex_weight
        
        # Optional: Hard thresholding to remove low-amplitude noise
        if self.threshold > 0:
            magnitude = torch.abs(x_filtered)
            mask = magnitude > self.threshold
            x_filtered = x_filtered * mask.float()
        
        # Apply inverse FFT to return to spatial domain
        output = torch.fft.irfft2(x_filtered, s=(H, W), dim=(-2, -1), norm="ortho")
        
        return output


class FrequencyLoss(nn.Module):
    """
    Frequency domain loss for enforcing edge sharpness.
    Computes L2 distance between FFT of prediction and ground truth.
    """
    
    def __init__(self, weight: float = 0.1):
        """
        Initialize FrequencyLoss.
        
        Args:
            weight: Weight factor for frequency loss in combined loss
        """
        super().__init__()
        self.weight = weight
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Compute frequency domain loss.
        
        Args:
            pred: Prediction tensor of shape (B, C, H, W)
            target: Ground truth tensor of shape (B, C, H, W)
            
        Returns:
            Scalar loss value
        """
        # Apply FFT
        pred_freq = torch.fft.rfft2(pred, dim=(-2, -1), norm="ortho")
        target_freq = torch.fft.rfft2(target, dim=(-2, -1), norm="ortho")
        
        # Compute L2 distance in frequency domain
        # Using both magnitude and phase information
        loss_real = F.mse_loss(pred_freq.real, target_freq.real)
        loss_imag = F.mse_loss(pred_freq.imag, target_freq.imag)
        
        return loss_real + loss_imag




### mamba_block.py


In [ ]:
"""
Mamba-like Block Module (Lightweight VSS Block Implementation).

Pure PyTorch implementation of Visual State Space (VSS) blocks for 2D image
processing without external SSM libraries. Uses GRU-based sequential scanning
to simulate selective state space dynamics.

Architecture:
    - DirectionalScanner: Multi-directional sequential scanning (4 directions)
    - VSSBlock: Core Mamba-like component with residual connections
    - MambaBlockStack: Hierarchical stacking of VSS blocks

References:
    [1] Gu & Dao, "Mamba: Linear-Time Sequence Modeling with Selective
        State Spaces," arXiv, 2023.
    [2] Liu et al., "VMamba: Visual State Space Model," arXiv, 2024.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Tuple, Optional


class DepthwiseSeparableConv2d(nn.Module):
    """Depthwise separable convolution block."""
    
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int = 3):
        super().__init__()
        self.depthwise = nn.Conv2d(
            in_channels, in_channels, kernel_size=kernel_size,
            padding=kernel_size // 2, groups=in_channels, bias=False
        )
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=True)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x


class DirectionalScanner(nn.Module):
    """
    Scans 2D feature maps in multiple directions to simulate 2D-SS2D.
    Implements sequential scanning in 4 directions: right, down, left, up.
    """
    
    def __init__(self, channels: int, scan_dim: int = 64):
        """
        Initialize DirectionalScanner.
        
        Args:
            channels: Number of input channels
            scan_dim: Hidden dimension for sequential processing
        """
        super().__init__()
        self.channels = channels
        self.scan_dim = scan_dim
        
        # Learnable projection to scan_dim for each direction
        self.proj_in = nn.Linear(channels, scan_dim)
        
        # GRU cell for sequential state processing (simulates SSM)
        self.gru_cell = nn.GRUCell(scan_dim, scan_dim)
        
        # Project back to original channels
        self.proj_out = nn.Linear(scan_dim, channels)
        
    def _scan_direction(self, x: torch.Tensor, direction: str) -> torch.Tensor:
        """
        Scan feature map in a specific direction.
        
        Args:
            x: Input tensor of shape (B, C, H, W)
            direction: One of ['right', 'down', 'left', 'up']
            
        Returns:
            Scanned tensor of shape (B, C, H, W)
        """
        B, C, H, W = x.shape
        
        # Prepare sequence based on direction
        if direction == "right":
            # Scan left-to-right: (B, H*W, C) after reshape
            x = x.permute(0, 2, 3, 1).reshape(B * H, W, C)  # (B*H, W, C)
        elif direction == "down":
            # Scan top-to-bottom
            x = x.permute(0, 3, 2, 1).reshape(B * W, H, C)  # (B*W, H, C)
        elif direction == "left":
            # Scan right-to-left (reverse)
            x = x.permute(0, 2, 3, 1).flip(1).reshape(B * H, W, C)  # (B*H, W, C)
        elif direction == "up":
            # Scan bottom-to-top (reverse)
            x = x.permute(0, 3, 2, 1).flip(1).reshape(B * W, H, C)  # (B*W, H, C)
        else:
            raise ValueError(f"Unknown direction: {direction}")
        
        # Project to scan dimension
        x = self.proj_in(x)  # (*, W/H, scan_dim)
        
        # Apply GRU cell sequentially (simulates SSM forward pass)
        outputs = []
        h = torch.zeros(x.shape[0], self.scan_dim, device=x.device, dtype=x.dtype)
        
        for t in range(x.shape[1]):
            h = self.gru_cell(x[:, t], h)  # GRU step
            outputs.append(h)
        
        x = torch.stack(outputs, dim=1)  # (*, W/H, scan_dim)
        
        # Project back to original channels
        x = self.proj_out(x)  # (*, W/H, C)
        
        # Reshape back to (B, C, H, W)
        if direction == "right":
            x = x.reshape(B, H, W, C).permute(0, 3, 1, 2)
        elif direction == "down":
            x = x.reshape(B, W, H, C).permute(0, 3, 2, 1)
        elif direction == "left":
            x = x.reshape(B, H, W, C).permute(0, 3, 1, 2).flip(-1)
        elif direction == "up":
            x = x.reshape(B, W, H, C).permute(0, 3, 2, 1).flip(-2)
        
        return x
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Apply directional scanning in 4 directions and aggregate.
        
        Args:
            x: Input tensor of shape (B, C, H, W)
            
        Returns:
            Aggregated output tensor of shape (B, C, H, W)
        """
        # Scan in all 4 directions
        scan_right = self._scan_direction(x, "right")
        scan_down = self._scan_direction(x, "down")
        scan_left = self._scan_direction(x, "left")
        scan_up = self._scan_direction(x, "up")
        
        # Aggregate by averaging
        output = (scan_right + scan_down + scan_left + scan_up) / 4.0
        
        return output


class VSSBlock(nn.Module):
    """
    Visual State Space Block - Core Mamba-like component.
    Combines convolutional preprocessing with multi-directional scanning.
    """
    
    def __init__(self, channels: int, hidden_dim: Optional[int] = None, 
                 scan_dim: int = 64, expansion_ratio: float = 2.0):
        """
        Initialize VSSBlock.
        
        Args:
            channels: Number of input channels
            hidden_dim: Hidden dimension (default: channels * expansion_ratio)
            scan_dim: Hidden dimension for directional scanner
            expansion_ratio: Channel expansion ratio for internal processing
        """
        super().__init__()
        self.channels = channels
        hidden_dim = hidden_dim or int(channels * expansion_ratio)
        
        # Preprocessing: expand channels
        self.norm1 = nn.GroupNorm(num_groups=32, num_channels=channels, eps=1e-6)
        self.conv_expand = nn.Conv2d(channels, hidden_dim, kernel_size=1, bias=True)
        
        # Directional scanning
        self.scanner = DirectionalScanner(hidden_dim, scan_dim=scan_dim)
        
        # Postprocessing: contract channels back
        self.norm2 = nn.GroupNorm(num_groups=32, num_channels=hidden_dim, eps=1e-6)
        self.conv_contract = nn.Conv2d(hidden_dim, channels, kernel_size=1, bias=True)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass with residual connection.
        
        Args:
            x: Input tensor of shape (B, C, H, W)
            
        Returns:
            Output tensor of shape (B, C, H, W) with residual
        """
        residual = x
        
        # Preprocessing
        x = self.norm1(x)
        x = self.conv_expand(x)
        x = F.gelu(x)
        
        # Directional scanning (core SSM-like operation)
        x = self.scanner(x)
        
        # Postprocessing
        x = self.norm2(x)
        x = self.conv_contract(x)
        
        # Residual connection
        output = x + residual
        
        return output


class MambaBlockStack(nn.Module):
    """Stack of multiple VSS blocks for hierarchical processing."""
    
    def __init__(self, channels: int, depth: int = 2, **kwargs):
        """
        Initialize stack of VSS blocks.
        
        Args:
            channels: Number of channels
            depth: Number of VSS blocks to stack
            **kwargs: Additional arguments passed to VSSBlock
        """
        super().__init__()
        self.blocks = nn.ModuleList([
            VSSBlock(channels, **kwargs) for _ in range(depth)
        ])
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for block in self.blocks:
            x = block(x)
        return x




### constellation_head.py


In [ ]:
"""
RBF Constellation Head for Semantic Segmentation.

Implements a Gaussian/RBF classifier inspired by digital modulation (PSK/QAM),
projecting features to a 2D I/Q space and measuring distance to fixed prototypes.

Key Concepts:
    1. Project features to 2D embedding space (I/Q plane)
    2. Fix class prototypes at constellation points (PSK/QPSK/8-PSK)
    3. Classify based on RBF kernel: P(y|x) ∝ exp(-γ||z - c||²)
    4. Output logits = -γ × squared_distance (for CrossEntropyLoss)

Advantages over standard Conv1x1 classifier:
    - Maximizes noise margin (decision boundaries equidistant from prototypes)
    - Geometric interpretation of class relationships
    - Robust to gradient explosion (bounded embedding space via Tanh)

References:
    [1] Proakis, "Digital Communications" - PSK/QAM constellation theory
    [2] RBF Networks - Gaussian kernel classifiers
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Tuple, Optional


class RBFConstellationHead(nn.Module):
    """
    RBF Constellation Head with Gaussian Classifier.
    
    Projects feature map to 2D latent space (I/Q plane) and computes
    class probabilities based on distance to fixed constellation prototypes.
    
    Architecture:
        1. Projector: Conv 3x3 → SiLU → Conv 1x1 → Tanh (→ [-1, 1])
        2. Prototypes: Fixed PSK constellation points (non-trainable)
        3. RBF Kernel: exp(-γ||z - c||²)
        4. Logits: -γ × ||z - c||² (for CrossEntropyLoss)
    
    Args:
        in_channels: Number of input feature channels
        num_classes: Number of output classes (determines constellation type)
        embedding_dim: Dimension of I/Q embedding (default 2 for 2D plane)
        init_gamma: Initial temperature parameter (γ = 1/2σ²)
    """
    
    def __init__(self, in_channels: int, num_classes: int = 4,
                 embedding_dim: int = 2, init_gamma: float = 1.0):
        super().__init__()
        
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.embedding_dim = embedding_dim
        
        # Feature projector: maps to 2D I/Q space
        mid_channels = in_channels // 2
        # Ensure num_groups divides num_channels (using GCD with 32)
        num_groups = math.gcd(mid_channels, 32)
        
        self.projector = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.GroupNorm(num_groups, mid_channels),
            nn.SiLU(),
            nn.Conv2d(in_channels // 2, embedding_dim, kernel_size=1, bias=True),
            nn.Tanh()  # Constrain to [-1, 1] for stable training
        )
        
        # Fixed constellation prototypes (non-trainable)
        prototypes = self._generate_constellation_points(num_classes, embedding_dim)
        self.register_buffer('prototypes', prototypes)
        
        # Learnable temperature (γ = 1 / 2σ²)
        # Higher γ = narrower Gaussian = harder boundaries
        self.gamma = nn.Parameter(torch.tensor(init_gamma))
    
    def _generate_constellation_points(self, num_classes: int, 
                                        embedding_dim: int) -> torch.Tensor:
        """
        Generate constellation points based on number of classes.
        
        Uses M-PSK (Phase Shift Keying) for 2D:
            - 2 classes: BPSK (±1)
            - 4 classes: QPSK (±1, ±j)
            - 8 classes: 8-PSK (unit circle, 45° spacing)
            - Other: uniform on unit circle
        
        Args:
            num_classes: Number of constellation points
            embedding_dim: Embedding dimension (2 for I/Q)
            
        Returns:
            Tensor of shape (embedding_dim, num_classes)
        """
        if embedding_dim != 2:
            # For higher dimensions, use random unit vectors
            points = torch.randn(embedding_dim, num_classes)
            points = F.normalize(points, dim=0)
            return points
        
        # Generate M-PSK constellation (2D)
        angles = torch.linspace(0, 2 * math.pi, num_classes + 1)[:-1]
        
        # Unit circle points
        points = torch.stack([
            torch.cos(angles),  # I component
            torch.sin(angles)   # Q component
        ], dim=0)  # (2, num_classes)
        
        return points
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Forward pass.
        
        Args:
            x: Feature map (B, C, H, W)
            
        Returns:
            Tuple of:
            - logits: Class logits (B, num_classes, H, W)
            - embeddings: 2D embeddings (B, 2, H, W)
        """
        B, C, H, W = x.shape
        
        # Project to 2D embedding space
        z = self.projector(x)  # (B, 2, H, W)
        
        # Reshape for distance computation
        # z: (B, 2, H, W) → (B, H, W, 2, 1)
        z_flat = z.permute(0, 2, 3, 1).unsqueeze(-1)  # (B, H, W, 2, 1)
        
        # prototypes: (2, num_classes) → (1, 1, 1, 2, num_classes)
        proto = self.prototypes.view(1, 1, 1, self.embedding_dim, self.num_classes)
        
        # Compute squared Euclidean distance: ||z - c||²
        # (B, H, W, 2, 1) - (1, 1, 1, 2, M) → (B, H, W, 2, M)
        diff = z_flat - proto
        squared_dist = torch.sum(diff ** 2, dim=3)  # (B, H, W, num_classes)
        
        # RBF/Gaussian logits: -γ × ||z - c||²
        # Higher γ = stricter classification
        gamma = F.softplus(self.gamma)  # Ensure positive
        logits = -gamma * squared_dist  # (B, H, W, num_classes)
        
        # Rearrange to (B, num_classes, H, W)
        logits = logits.permute(0, 3, 1, 2)
        
        return logits, z
    
    def get_noise_margin(self) -> float:
        """
        Compute minimum distance between constellation points (noise margin).
        
        Returns:
            Minimum Euclidean distance between any two prototypes
        """
        with torch.no_grad():
            # prototypes: (2, M)
            proto = self.prototypes.T  # (M, 2)
            
            # Pairwise distances
            dist_matrix = torch.cdist(proto, proto)
            
            # Mask diagonal
            mask = torch.eye(self.num_classes, device=proto.device).bool()
            dist_matrix = dist_matrix.masked_fill(mask, float('inf'))
            
            min_dist = dist_matrix.min().item()
            
        return min_dist


class ConstellationVisualization:
    """Helper class for visualizing constellation embeddings."""
    
    @staticmethod
    def plot_constellation(embeddings: torch.Tensor, labels: torch.Tensor,
                          prototypes: torch.Tensor, save_path: Optional[str] = None):
        """
        Plot 2D constellation with class prototypes.
        
        Args:
            embeddings: Tensor of shape (N, 2) with embedded points
            labels: Tensor of shape (N,) with class labels
            prototypes: Tensor of shape (2, num_classes) with prototypes
            save_path: Optional path to save figure
        """
        try:
            import matplotlib.pyplot as plt
        except ImportError:
            print("matplotlib not available for visualization")
            return
        
        embeddings = embeddings.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()
        prototypes = prototypes.detach().cpu().numpy().T  # (num_classes, 2)
        
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
        
        # Plot embeddings colored by class
        num_classes = prototypes.shape[0]
        colors = plt.cm.tab10(range(num_classes))
        
        for cls in range(num_classes):
            mask = labels == cls
            if mask.sum() > 0:
                ax.scatter(embeddings[mask, 0], embeddings[mask, 1],
                          c=[colors[cls]], alpha=0.5, s=10, label=f'Class {cls}')
        
        # Plot prototypes as larger markers
        for cls in range(num_classes):
            ax.scatter(prototypes[cls, 0], prototypes[cls, 1],
                      c=[colors[cls]], s=200, marker='*', edgecolors='black',
                      linewidth=2)
        
        # Draw unit circle (constellation reference)
        theta = torch.linspace(0, 2 * math.pi, 100)
        ax.plot(torch.cos(theta).numpy(), torch.sin(theta).numpy(),
               'k--', alpha=0.3, label='Unit Circle')
        
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)
        ax.set_aspect('equal')
        ax.set_xlabel('I (In-Phase)')
        ax.set_ylabel('Q (Quadrature)')
        ax.set_title('Constellation Diagram')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()


# =============================================================================
# Testing
# =============================================================================



### gabor_implicit.py


In [ ]:
"""
Gabor Implicit Neural Representation Module

Replaces standard Fourier Features with Gabor Basis for implicit neural representations.

Key insight: 
- Fourier (sin/cos): Oscillates infinitely → causes Gibbs ringing artifacts
- Gabor (Gaussian × sin): Localized oscillation → clean boundaries without ringing

This implements the "Fine Branch" of the EGM-Net architecture.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Optional, Tuple, List


class GaborBasis(nn.Module):
    """
    Gabor Basis Functions for coordinate encoding.
    
    Gabor function: g(x) = exp(-x²/2σ²) × cos(2πfx + φ)
    
    Unlike Fourier features which oscillate infinitely, Gabor wavelets
    are spatially localized, preventing Gibbs phenomenon (ringing artifacts).
    """
    
    def __init__(self, input_dim: int = 2, num_frequencies: int = 64,
                 sigma_range: Tuple[float, float] = (0.1, 2.0),
                 freq_range: Tuple[float, float] = (1.0, 10.0),
                 learnable: bool = True):
        """
        Initialize Gabor Basis.
        
        Args:
            input_dim: Dimension of input coordinates (2 for images)
            num_frequencies: Number of Gabor basis functions
            sigma_range: Range of Gaussian envelope widths
            freq_range: Range of oscillation frequencies
            learnable: Whether parameters are learnable
        """
        super().__init__()
        self.input_dim = input_dim
        self.num_frequencies = num_frequencies
        self.output_dim = num_frequencies * 2  # sin and cos components
        
        # Initialize frequencies uniformly in log space
        log_freqs = torch.linspace(
            math.log(freq_range[0]), 
            math.log(freq_range[1]), 
            num_frequencies
        )
        freqs = torch.exp(log_freqs)
        
        # Initialize sigmas (Gaussian envelope widths)
        sigmas = torch.linspace(sigma_range[0], sigma_range[1], num_frequencies)
        
        # Random orientations for 2D
        orientations = torch.rand(num_frequencies) * 2 * math.pi
        
        # Random phases
        phases = torch.rand(num_frequencies) * 2 * math.pi
        
        # Create direction vectors from orientations
        directions = torch.stack([
            torch.cos(orientations),
            torch.sin(orientations)
        ], dim=-1)  # (num_freq, 2)
        
        if learnable:
            self.freqs = nn.Parameter(freqs)
            self.sigmas = nn.Parameter(sigmas)
            self.directions = nn.Parameter(directions)
            self.phases = nn.Parameter(phases)
        else:
            self.register_buffer('freqs', freqs)
            self.register_buffer('sigmas', sigmas)
            self.register_buffer('directions', directions)
            self.register_buffer('phases', phases)
    
    def forward(self, coords: torch.Tensor) -> torch.Tensor:
        """
        Encode coordinates using Gabor basis.
        
        Args:
            coords: Coordinate tensor of shape (..., input_dim)
            
        Returns:
            Gabor features of shape (..., output_dim)
        """
        # Normalize directions
        directions = F.normalize(self.directions, dim=-1)  # (num_freq, 2)
        
        # Project coordinates onto directions
        # coords: (..., 2), directions: (num_freq, 2)
        proj = torch.matmul(coords, directions.T)  # (..., num_freq)
        
        # Compute Gaussian envelope
        # exp(-proj² / (2σ²))
        sigmas = torch.abs(self.sigmas) + 0.01  # Ensure positive
        gaussian = torch.exp(-proj**2 / (2 * sigmas**2 + 1e-8))
        
        # Compute oscillatory component
        # cos(2πf·proj + φ), sin(2πf·proj + φ)
        freqs = torch.abs(self.freqs) + 0.1  # Ensure positive
        arg = 2 * math.pi * freqs * proj + self.phases
        
        cos_comp = gaussian * torch.cos(arg)
        sin_comp = gaussian * torch.sin(arg)
        
        # Concatenate sin and cos
        gabor_features = torch.cat([cos_comp, sin_comp], dim=-1)
        
        return gabor_features


class FourierFeatures(nn.Module):
    """
    Standard Fourier Features for comparison.
    
    From "Fourier Features Let Networks Learn High Frequency Functions"
    """
    
    def __init__(self, input_dim: int = 2, num_frequencies: int = 64,
                 scale: float = 10.0, learnable: bool = False):
        """
        Initialize Fourier Features.
        
        Args:
            input_dim: Dimension of input coordinates
            num_frequencies: Number of frequency bands
            scale: Standard deviation for frequency sampling
            learnable: Whether B matrix is learnable
        """
        super().__init__()
        self.input_dim = input_dim
        self.num_frequencies = num_frequencies
        self.output_dim = num_frequencies * 2
        
        # Random frequency matrix
        B = torch.randn(input_dim, num_frequencies) * scale
        
        if learnable:
            self.B = nn.Parameter(B)
        else:
            self.register_buffer('B', B)
    
    def forward(self, coords: torch.Tensor) -> torch.Tensor:
        """
        Encode coordinates using Fourier features.
        
        Args:
            coords: Coordinate tensor of shape (..., input_dim)
            
        Returns:
            Fourier features of shape (..., output_dim)
        """
        # Project: coords @ B
        proj = 2 * math.pi * torch.matmul(coords, self.B)  # (..., num_freq)
        
        # Sin and cos
        return torch.cat([torch.cos(proj), torch.sin(proj)], dim=-1)


class SIRENLayer(nn.Module):
    """
    SIREN (Sinusoidal Representation Networks) layer.
    
    Uses periodic sine activation instead of ReLU.
    From "Implicit Neural Representations with Periodic Activation Functions"
    """
    
    def __init__(self, in_features: int, out_features: int, 
                 omega_0: float = 30.0, is_first: bool = False):
        """
        Initialize SIREN layer.
        
        Args:
            in_features: Number of input features
            out_features: Number of output features
            omega_0: Frequency multiplier (ω₀ in paper)
            is_first: Whether this is the first layer (uses different init)
        """
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.in_features = in_features
        
        self.linear = nn.Linear(in_features, out_features)
        self._init_weights()
    
    def _init_weights(self):
        """SIREN-specific weight initialization."""
        with torch.no_grad():
            if self.is_first:
                # First layer: uniform in [-1/n, 1/n]
                self.linear.weight.uniform_(-1 / self.in_features, 
                                            1 / self.in_features)
            else:
                # Other layers: uniform in [-sqrt(6/n)/ω₀, sqrt(6/n)/ω₀]
                bound = math.sqrt(6 / self.in_features) / self.omega_0
                self.linear.weight.uniform_(-bound, bound)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.sin(self.omega_0 * self.linear(x))


class GaborNet(nn.Module):
    """
    GaborNet: MLP with Gabor basis input encoding.
    
    Architecture: Gabor Encoding → SIREN Layers → Output
    
    This replaces standard MLP/KAN for implicit neural representations,
    providing better stability and localized high-frequency learning.
    """
    
    def __init__(self, coord_dim: int = 2, feature_dim: int = 256,
                 hidden_dim: int = 256, output_dim: int = 1,
                 num_layers: int = 4, num_frequencies: int = 64,
                 use_gabor: bool = True, omega_0: float = 30.0):
        """
        Initialize GaborNet.
        
        Args:
            coord_dim: Dimension of input coordinates (2 for images)
            feature_dim: Dimension of input features from encoder
            hidden_dim: Hidden layer dimension
            output_dim: Output dimension (num_classes for segmentation)
            num_layers: Number of SIREN layers
            num_frequencies: Number of Gabor/Fourier frequencies
            use_gabor: Use Gabor (True) or Fourier (False) features
            omega_0: SIREN frequency parameter
        """
        super().__init__()
        
        # Coordinate encoding
        if use_gabor:
            self.coord_encoder = GaborBasis(
                input_dim=coord_dim,
                num_frequencies=num_frequencies,
                learnable=True
            )
        else:
            self.coord_encoder = FourierFeatures(
                input_dim=coord_dim,
                num_frequencies=num_frequencies,
                learnable=False
            )
        
        coord_encoded_dim = self.coord_encoder.output_dim
        
        # Input dimension: encoded coords + features
        input_dim = coord_encoded_dim + feature_dim
        
        # Build SIREN network
        layers = []
        
        # First layer
        layers.append(SIRENLayer(input_dim, hidden_dim, omega_0, is_first=True))
        
        # Hidden layers
        for _ in range(num_layers - 2):
            layers.append(SIRENLayer(hidden_dim, hidden_dim, omega_0, is_first=False))
        
        # Final layer (linear, no sine activation)
        layers.append(nn.Linear(hidden_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, coords: torch.Tensor, features: torch.Tensor) -> torch.Tensor:
        """
        Forward pass.
        
        Args:
            coords: Normalized coordinates of shape (B, N, 2) in [-1, 1]
            features: Features from encoder of shape (B, N, feature_dim)
            
        Returns:
            Output of shape (B, N, output_dim)
        """
        # Encode coordinates
        coord_encoded = self.coord_encoder(coords)  # (B, N, coord_encoded_dim)
        
        # Concatenate with features
        x = torch.cat([coord_encoded, features], dim=-1)  # (B, N, input_dim)
        
        # Pass through network
        output = self.network(x)  # (B, N, output_dim)
        
        return output


class ImplicitSegmentationHead(nn.Module):
    """
    Implicit Segmentation Head for continuous boundary representation.
    
    Takes feature maps from encoder and outputs continuous segmentation
    at arbitrary resolution using Gabor-based implicit representation.
    """
    
    def __init__(self, feature_channels: int = 64, num_classes: int = 2,
                 hidden_dim: int = 256, num_layers: int = 4,
                 num_frequencies: int = 64, use_gabor: bool = True):
        """
        Initialize Implicit Segmentation Head.
        
        Args:
            feature_channels: Number of channels in input feature map
            num_classes: Number of segmentation classes
            hidden_dim: Hidden dimension of GaborNet
            num_layers: Number of layers in GaborNet
            num_frequencies: Number of Gabor frequencies
            use_gabor: Use Gabor (True) or Fourier (False) basis
        """
        super().__init__()
        
        self.feature_channels = feature_channels
        self.num_classes = num_classes
        
        # Feature projector (reduce channel dimension)
        self.feature_proj = nn.Sequential(
            nn.Conv2d(feature_channels, hidden_dim, kernel_size=1),
            nn.GroupNorm(32, hidden_dim),
            nn.GELU()
        )
        
        # Implicit decoder
        self.implicit_decoder = GaborNet(
            coord_dim=2,
            feature_dim=hidden_dim,
            hidden_dim=hidden_dim,
            output_dim=num_classes,
            num_layers=num_layers,
            num_frequencies=num_frequencies,
            use_gabor=use_gabor
        )
    
    def sample_features(self, feature_map: torch.Tensor, 
                       coords: torch.Tensor) -> torch.Tensor:
        """
        Sample features at given coordinates using bilinear interpolation.
        
        Args:
            feature_map: Feature tensor of shape (B, C, H, W)
            coords: Coordinates of shape (B, N, 2) in [-1, 1]
            
        Returns:
            Sampled features of shape (B, N, C)
        """
        B, C, H, W = feature_map.shape
        N = coords.shape[1]
        
        # Reshape coords for grid_sample: (B, N, 1, 2) -> (B, 1, N, 2)
        # grid_sample expects (B, H, W, 2) where last dim is (x, y)
        grid = coords.view(B, 1, N, 2)
        
        # Sample using bilinear interpolation
        # feature_map: (B, C, H, W), grid: (B, 1, N, 2)
        # output: (B, C, 1, N)
        sampled = F.grid_sample(
            feature_map, grid,
            mode='bilinear',
            padding_mode='border',
            align_corners=True
        )
        
        # Reshape: (B, C, 1, N) -> (B, N, C)
        sampled = sampled.squeeze(2).permute(0, 2, 1)
        
        return sampled
    
    def forward(self, feature_map: torch.Tensor, 
                coords: Optional[torch.Tensor] = None,
                output_size: Optional[Tuple[int, int]] = None) -> torch.Tensor:
        """
        Forward pass.
        
        Args:
            feature_map: Feature tensor of shape (B, C, H, W)
            coords: Optional query coordinates of shape (B, N, 2) in [-1, 1]
                   If None, generates grid based on output_size
            output_size: Optional (H, W) for output resolution
                        If None and coords is None, uses feature_map size * 4
            
        Returns:
            Segmentation logits of shape (B, num_classes, H_out, W_out) or (B, N, num_classes)
        """
        B, C, H_feat, W_feat = feature_map.shape
        device = feature_map.device
        
        # Project features
        feature_map = self.feature_proj(feature_map)  # (B, hidden_dim, H, W)
        
        # Generate coordinates if not provided
        if coords is None:
            if output_size is None:
                output_size = (H_feat * 4, W_feat * 4)
            
            H_out, W_out = output_size
            
            # Create normalized coordinate grid [-1, 1]
            y = torch.linspace(-1, 1, H_out, device=device)
            x = torch.linspace(-1, 1, W_out, device=device)
            yy, xx = torch.meshgrid(y, x, indexing='ij')
            coords = torch.stack([xx, yy], dim=-1)  # (H_out, W_out, 2)
            coords = coords.view(1, -1, 2).expand(B, -1, -1)  # (B, H*W, 2)
            
            reshape_output = True
        else:
            reshape_output = False
            H_out, W_out = None, None
        
        # Sample features at coordinates
        features = self.sample_features(feature_map, coords)  # (B, N, hidden_dim)
        
        # Implicit decoding
        logits = self.implicit_decoder(coords, features)  # (B, N, num_classes)
        
        # Reshape to image if using grid
        if reshape_output:
            logits = logits.view(B, H_out, W_out, self.num_classes)
            logits = logits.permute(0, 3, 1, 2)  # (B, C, H, W)
        
        return logits


class FiLMLayer(nn.Module):
    """
    Feature-wise Linear Modulation (FiLM) Layer.
    
    Applies affine transformation conditioned on input features:
        output = γ(features) × input + β(features)
    
    This allows the network to dynamically adjust activations based on context.
    
    Args:
        feature_dim: Dimension of conditioning features
        modulation_dim: Dimension of signal to modulate
    """
    
    def __init__(self, feature_dim: int, modulation_dim: int):
        super().__init__()
        
        # Predict scale (γ) and shift (β) from features
        self.gamma_proj = nn.Linear(feature_dim, modulation_dim)
        self.beta_proj = nn.Linear(feature_dim, modulation_dim)
        
        # Initialize to identity transform
        nn.init.ones_(self.gamma_proj.weight.data[:modulation_dim, :modulation_dim // feature_dim + 1])
        nn.init.zeros_(self.gamma_proj.bias.data)
        nn.init.zeros_(self.beta_proj.weight.data)
        nn.init.zeros_(self.beta_proj.bias.data)
    
    def forward(self, x: torch.Tensor, condition: torch.Tensor) -> torch.Tensor:
        """
        Apply FiLM modulation.
        
        Args:
            x: Input tensor to modulate (..., modulation_dim)
            condition: Conditioning features (..., feature_dim)
            
        Returns:
            Modulated tensor (..., modulation_dim)
        """
        gamma = self.gamma_proj(condition)  # Scale
        beta = self.beta_proj(condition)    # Shift
        
        return gamma * x + beta


class EnergyGatedGaborImplicit(nn.Module):
    """
    Energy-Gated Gabor Implicit Neural Representation for Fine Segmentation.
    
    Key features:
    1. Gabor/Raised Cosine basis encoding (prevents Gibbs ringing)
    2. FiLM conditioning (feature-dependent modulation)
    3. Physics Gating (output × energy_map to suppress flat regions)
    
    The energy gating ensures:
    - High energy (boundaries): Full implicit output for sharp details
    - Low energy (flat): Suppressed output to avoid artifacts
    
    Architecture:
        coords → Gabor Encoding → FiLM(features) → MLP → × Energy → Output
    
    Args:
        feature_dim: Dimension of input features
        num_classes: Number of output classes
        hidden_dim: Hidden layer dimension
        num_layers: Number of MLP layers
        num_frequencies: Number of Gabor basis functions
        use_gabor: Use Gabor (True) or Fourier (False) basis
        omega_0: SIREN frequency parameter
    """
    
    def __init__(self, feature_dim: int, num_classes: int,
                 hidden_dim: int = 256, num_layers: int = 4,
                 num_frequencies: int = 64, use_gabor: bool = True,
                 omega_0: float = 30.0):
        super().__init__()
        
        self.feature_dim = feature_dim
        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        
        # Coordinate encoder (Gabor or Fourier)
        if use_gabor:
            self.coord_encoder = GaborBasis(
                input_dim=2, num_frequencies=num_frequencies, learnable=True
            )
        else:
            self.coord_encoder = FourierFeatures(
                input_dim=2, num_frequencies=num_frequencies, learnable=False
            )
        
        coord_encoded_dim = self.coord_encoder.output_dim
        
        # FiLM layers for feature conditioning
        self.film_layers = nn.ModuleList([
            FiLMLayer(feature_dim, hidden_dim)
            for _ in range(num_layers - 1)
        ])
        
        # MLP decoder with SIREN layers
        self.input_proj = SIRENLayer(coord_encoded_dim, hidden_dim, omega_0, is_first=True)
        
        self.hidden_layers = nn.ModuleList([
            SIRENLayer(hidden_dim, hidden_dim, omega_0, is_first=False)
            for _ in range(num_layers - 2)
        ])
        
        # Output layer (linear, no sine)
        self.output_proj = nn.Linear(hidden_dim, num_classes)
        
        # Energy gating parameters
        self.gate_scale = nn.Parameter(torch.ones(1))
        self.gate_bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, coords: torch.Tensor, features: torch.Tensor,
                energy: torch.Tensor) -> torch.Tensor:
        """
        Forward pass with FiLM conditioning and energy gating.
        
        Args:
            coords: Query coordinates (B, N, 2) in [-1, 1]
            features: Conditioning features (B, N, feature_dim)
            energy: Energy values at query points (B, N, 1) in [0, 1]
            
        Returns:
            Gated output logits (B, N, num_classes)
        """
        # 1. Encode coordinates with Gabor basis
        coord_enc = self.coord_encoder(coords)  # (B, N, coord_encoded_dim)
        
        # 2. Initial projection
        x = self.input_proj(coord_enc)  # (B, N, hidden_dim)
        
        # 3. Process through FiLM-modulated layers
        for i, (hidden_layer, film_layer) in enumerate(
            zip(self.hidden_layers, self.film_layers[:-1])
        ):
            # Apply SIREN layer
            x = hidden_layer(x)
            # Apply FiLM modulation
            x = film_layer(x, features)
        
        # Final FiLM modulation
        if len(self.film_layers) > 0:
            x = self.film_layers[-1](x, features)
        
        # 4. Output projection
        logits = self.output_proj(x)  # (B, N, num_classes)
        
        # 5. Physics Gating
        # Scale energy with learnable parameters
        gate = torch.sigmoid(energy * self.gate_scale + self.gate_bias)  # (B, N, 1)
        
        # Apply gating
        gated_logits = logits * gate  # (B, N, num_classes)
        
        return gated_logits


class EnergyGatedImplicitHead(nn.Module):
    """
    Complete Energy-Gated Implicit Segmentation Head.
    
    Wraps EnergyGatedGaborImplicit with feature sampling from encoder output.
    
    Args:
        feature_channels: Number of channels in input feature map
        num_classes: Number of output classes
        hidden_dim: Hidden dimension
        num_layers: Number of MLP layers
        num_frequencies: Number of Gabor frequencies
    """
    
    def __init__(self, feature_channels: int, num_classes: int,
                 hidden_dim: int = 256, num_layers: int = 4,
                 num_frequencies: int = 64):
        super().__init__()
        
        self.num_classes = num_classes
        
        # Feature projection
        self.feature_proj = nn.Sequential(
            nn.Conv2d(feature_channels, hidden_dim, kernel_size=1),
            nn.GroupNorm(min(32, hidden_dim), hidden_dim),
            nn.GELU()
        )
        
        # Implicit decoder with energy gating
        self.implicit_decoder = EnergyGatedGaborImplicit(
            feature_dim=hidden_dim,
            num_classes=num_classes,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            num_frequencies=num_frequencies,
            use_gabor=True
        )
    
    def sample_at_coords(self, feature_map: torch.Tensor,
                        energy_map: torch.Tensor,
                        coords: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Sample features and energy at given coordinates.
        
        Args:
            feature_map: (B, C, H, W)
            energy_map: (B, 1, H, W)
            coords: (B, N, 2) in [-1, 1]
            
        Returns:
            Tuple of (features, energy) both of shape (B, N, C)
        """
        B, C, H, W = feature_map.shape
        N = coords.shape[1]
        
        grid = coords.view(B, 1, N, 2)
        
        # Sample features
        features = F.grid_sample(
            feature_map, grid, mode='bilinear',
            padding_mode='border', align_corners=True
        ).squeeze(2).permute(0, 2, 1)  # (B, N, C)
        
        # Sample energy
        energy = F.grid_sample(
            energy_map, grid, mode='bilinear',
            padding_mode='border', align_corners=True
        ).squeeze(2).permute(0, 2, 1)  # (B, N, 1)
        
        return features, energy
    
    def forward(self, feature_map: torch.Tensor,
                energy_map: torch.Tensor,
                coords: Optional[torch.Tensor] = None,
                output_size: Optional[Tuple[int, int]] = None) -> torch.Tensor:
        """
        Forward pass.
        
        Args:
            feature_map: Feature tensor (B, C, H, W)
            energy_map: Energy map (B, 1, H, W)
            coords: Optional query coordinates (B, N, 2)
            output_size: Optional output size (H, W)
            
        Returns:
            Segmentation logits (B, num_classes, H, W) or (B, N, num_classes)
        """
        B, C, H_feat, W_feat = feature_map.shape
        device = feature_map.device
        
        # Project features
        feature_map = self.feature_proj(feature_map)
        
        # Generate grid if no coords provided
        if coords is None:
            if output_size is None:
                output_size = (H_feat * 4, W_feat * 4)
            
            H_out, W_out = output_size
            
            y = torch.linspace(-1, 1, H_out, device=device)
            x = torch.linspace(-1, 1, W_out, device=device)
            yy, xx = torch.meshgrid(y, x, indexing='ij')
            coords = torch.stack([xx, yy], dim=-1).view(1, -1, 2).expand(B, -1, -1)
            
            reshape_output = True
        else:
            reshape_output = False
            H_out, W_out = None, None
        
        # Sample features and energy
        features, energy = self.sample_at_coords(feature_map, energy_map, coords)
        
        # Implicit decoding with energy gating
        logits = self.implicit_decoder(coords, features, energy)
        
        # Reshape if needed
        if reshape_output:
            logits = logits.view(B, H_out, W_out, self.num_classes)
            logits = logits.permute(0, 3, 1, 2)
        
        return logits



    print("Testing Gabor Implicit Modules...")
    
    # Test Gabor Basis
    print("\n[1] Testing GaborBasis...")
    gabor = GaborBasis(input_dim=2, num_frequencies=32)
    coords = torch.randn(4, 100, 2)  # (B, N, 2)
    encoded = gabor(coords)
    print(f"Input coords: {coords.shape}")
    print(f"Gabor encoded: {encoded.shape}")
    
    # Test GaborNet
    print("\n[2] Testing GaborNet...")
    net = GaborNet(coord_dim=2, feature_dim=64, hidden_dim=128, 
                   output_dim=3, num_layers=3, num_frequencies=32)
    features = torch.randn(4, 100, 64)
    output = net(coords, features)
    print(f"GaborNet output: {output.shape}")
    
    # Test ImplicitSegmentationHead
    print("\n[3] Testing ImplicitSegmentationHead...")
    head = ImplicitSegmentationHead(
        feature_channels=64, num_classes=3,
        hidden_dim=128, num_layers=3, num_frequencies=32
    )
    feature_map = torch.randn(2, 64, 32, 32)
    
    # Test with automatic grid
    seg_output = head(feature_map, output_size=(128, 128))
    print(f"Feature map: {feature_map.shape}")
    print(f"Segmentation output (grid): {seg_output.shape}")
    
    # Test with custom coordinates
    custom_coords = torch.rand(2, 500, 2) * 2 - 1  # Random points in [-1, 1]
    seg_points = head(feature_map, coords=custom_coords)
    print(f"Segmentation output (points): {seg_points.shape}")
    
    print("\n✓ All tests passed!")


### hrnet_mamba.py


In [ ]:
"""
HRNetV2-Mamba Backbone for EGM-Net.

A dual-stream architecture that maintains high-resolution features throughout,
replacing downsampling/upsampling with parallel multi-resolution streams.

Architecture Overview:
    1. Two parallel streams: High-Res (H/4) and Low-Res (H/8 → H/16)
    2. SpectralVSSBlock: Combines Mamba (spatial) + FFT (spectral) processing
    3. MultiScaleFusion: Periodic exchange of information between streams
    4. Aggregation: Upsample all to highest resolution and concatenate

Key Features:
    - High-res stream preserves boundary details (Energy, Frequency signals)
    - Low-res stream captures semantic context (Intensity)
    - No information loss from deep downsampling
    - Spectral Mamba blocks overcome spectral bias

References:
    [1] Wang et al., "Deep High-Resolution Representation Learning," CVPR 2020.
    [2] Liu et al., "VMamba: Visual State Space Model," arXiv 2024.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple, List, Dict
import math

# Handle imports for both package and standalone usage
try:
except (ImportError, ValueError):
    try:
    except ImportError:
        try:
            import sys
            import os
            # Add parent directory of 'models' (i.e., 'src') to path
            current_dir = os.path.dirname(os.path.abspath(__file__))
            src_dir = os.path.dirname(current_dir)
            if src_dir not in sys.path:
                sys.path.append(src_dir)
            
        except ImportError as e:
            raise ImportError(f"Could not import required modules (mamba_block, spectral_layers): {e}")


# =============================================================================
# Spectral VSS Block: Mamba + FFT Dual-Path Processing
# =============================================================================

class SpectralVSSBlock(nn.Module):
    """
    Spectral VSS Block combining spatial and frequency domain processing.
    
    Two parallel branches:
        A) Spatial Branch: Mamba (SS2D) for global context with O(N) complexity
        B) Spectral Branch: FFT → Learnable Gating → IFFT to overcome spectral bias
    
    The outputs are blended with a learnable weight.
    
    Args:
        channels: Number of input/output channels
        height: Feature map height (for spectral weights)
        width: Feature map width (for spectral weights)
        mamba_depth: Number of stacked Mamba blocks
        expansion_ratio: Channel expansion in Mamba blocks
        spectral_threshold: Threshold for spectral gating
    """
    
    def __init__(self, channels: int, height: int, width: int,
                 mamba_depth: int = 2, expansion_ratio: float = 2.0,
                 spectral_threshold: float = 0.1):
        super().__init__()
        
        self.channels = channels
        self.height = height
        self.width = width
        
        # Branch A: Spatial path (VSS/Mamba Blocks)
        self.vss_blocks = MambaBlockStack(
            channels, 
            depth=mamba_depth,
            expansion_ratio=expansion_ratio,
            scan_dim=min(64, channels)
        )
        
        # Branch B: Spectral path (FFT-based filtering)
        self.spectral_gate = SpectralGating(
            channels, height, width,
            threshold=spectral_threshold,
            complex_init="kaiming"
        )
        
        # Learnable fusion weight (σ(w) blends spatial and spectral)
        self.fusion_weight = nn.Parameter(torch.tensor(0.5))
        
        # Optional: Layer norm for stability
        self.norm = nn.GroupNorm(min(32, channels), channels)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass with dual-path processing.
        
        Args:
            x: Input tensor (B, C, H, W)
            
        Returns:
            Output tensor (B, C, H, W)
        """
        # Branch A: Spatial context via Mamba
        spatial_out = self.vss_blocks(x)
        
        # Branch B: Frequency filtering via FFT
        spectral_out = self.spectral_gate(x)
        
        # Learnable fusion
        weight = torch.sigmoid(self.fusion_weight)
        output = weight * spatial_out + (1 - weight) * spectral_out
        
        # Normalize
        output = self.norm(output)
        
        return output


# =============================================================================
# Multi-Scale Fusion Module
# =============================================================================

class MultiScaleFusion(nn.Module):
    """
    Multi-scale fusion between high-resolution and low-resolution streams.
    
    Enables bidirectional information flow:
        - High → Low: Strided Conv to downsample
        - Low → High: Bilinear Upsample + 1x1 Conv
    
    This allows:
        - Low-res stream to know "where" (from high-res)
        - High-res stream to know "what" (from low-res)
    
    Args:
        high_channels: Channels in high-res stream
        low_channels: Channels in low-res stream
        scale_factor: Spatial scale ratio between streams (e.g., 2)
    """
    
    def __init__(self, high_channels: int, low_channels: int, scale_factor: int = 2):
        super().__init__()
        
        self.scale_factor = scale_factor
        
        # High → Low: Downsample path
        self.high_to_low = nn.Sequential(
            nn.Conv2d(high_channels, low_channels, kernel_size=3, 
                     stride=scale_factor, padding=1, bias=False),
            nn.GroupNorm(min(32, low_channels), low_channels),
            nn.GELU()
        )
        
        # Low → High: Upsample path
        self.low_to_high = nn.Sequential(
            nn.Upsample(scale_factor=scale_factor, mode='bilinear', align_corners=True),
            nn.Conv2d(low_channels, high_channels, kernel_size=1, bias=False),
            nn.GroupNorm(min(32, high_channels), high_channels),
            nn.GELU()
        )
        
        # Fusion weights (learnable gating)
        self.high_gate = nn.Parameter(torch.ones(1) * 0.5)
        self.low_gate = nn.Parameter(torch.ones(1) * 0.5)
    
    def forward(self, high_feat: torch.Tensor, 
                low_feat: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Fuse features from both streams.
        
        Args:
            high_feat: High-res features (B, C_h, H, W)
            low_feat: Low-res features (B, C_l, H/s, W/s)
            
        Returns:
            Tuple of (updated_high, updated_low)
        """
        # High → Low contribution
        high_to_low = self.high_to_low(high_feat)
        
        # Low → High contribution
        low_to_high = self.low_to_high(low_feat)
        
        # Gated fusion
        h_gate = torch.sigmoid(self.high_gate)
        l_gate = torch.sigmoid(self.low_gate)
        
        # Update streams
        new_high = high_feat + h_gate * low_to_high
        new_low = low_feat + l_gate * high_to_low
        
        return new_high, new_low


# =============================================================================
# HRNet Stage: Single Resolution Processing Stage
# =============================================================================

class HRNetStage(nn.Module):
    """
    Single stage of HRNet processing at one resolution.
    
    Contains multiple SpectralVSSBlocks for deep feature extraction.
    
    Args:
        channels: Number of channels
        height: Feature map height
        width: Feature map width
        num_blocks: Number of SpectralVSSBlocks
        mamba_depth: Depth of Mamba blocks within each SpectralVSSBlock
    """
    
    def __init__(self, channels: int, height: int, width: int,
                 num_blocks: int = 2, mamba_depth: int = 2):
        super().__init__()
        
        self.blocks = nn.ModuleList([
            SpectralVSSBlock(channels, height, width, mamba_depth=mamba_depth)
            for _ in range(num_blocks)
        ])
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Process through all blocks."""
        for block in self.blocks:
            x = block(x)
        return x


# =============================================================================
# Stem: Initial Feature Extraction
# =============================================================================

class HRNetStem(nn.Module):
    """
    Initial feature extraction stem.
    
    Converts input image to initial feature representation with
    strided convolutions for efficiency.
    
    Args:
        in_channels: Number of input channels (3 for Intensity + Riesz)
        out_channels: Output feature channels
        stride: Total stride of stem (typically 4)
    """
    
    def __init__(self, in_channels: int = 3, out_channels: int = 64, stride: int = 4):
        super().__init__()
        
        # Two-stage striding: 2x each
        mid_channels = out_channels // 2
        
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=3, 
                              stride=2, padding=1, bias=False)
        self.norm1 = nn.GroupNorm(min(32, mid_channels), mid_channels)
        self.act1 = nn.GELU()
        
        self.conv2 = nn.Conv2d(mid_channels, out_channels, kernel_size=3,
                              stride=2, padding=1, bias=False)
        self.norm2 = nn.GroupNorm(min(32, out_channels), out_channels)
        self.act2 = nn.GELU()
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Extract initial features with stride-4 reduction."""
        x = self.act1(self.norm1(self.conv1(x)))
        x = self.act2(self.norm2(self.conv2(x)))
        return x


# =============================================================================
# Aggregation Layer: Combine All Resolutions
# =============================================================================

class AggregationLayer(nn.Module):
    """
    Aggregate multi-resolution features for output.
    
    Following HRNetV2: Upsample all resolutions to highest resolution
    and concatenate along channel dimension.
    
    Args:
        high_channels: Channels from high-res stream
        low_channels: Channels from low-res stream
        out_channels: Output channels after aggregation
        scale_factor: Scale factor to upsample low-res
    """
    
    def __init__(self, high_channels: int, low_channels: int, 
                 out_channels: int, scale_factor: int = 2):
        super().__init__()
        
        self.scale_factor = scale_factor
        
        # Upsample low-res to match high-res
        self.upsample_low = nn.Sequential(
            nn.Upsample(scale_factor=scale_factor, mode='bilinear', align_corners=True),
            nn.Conv2d(low_channels, low_channels, kernel_size=1, bias=False),
            nn.GroupNorm(min(32, low_channels), low_channels)
        )
        
        # Combine and project
        total_channels = high_channels + low_channels
        self.projection = nn.Sequential(
            nn.Conv2d(total_channels, out_channels, kernel_size=1, bias=False),
            nn.GroupNorm(min(32, out_channels), out_channels),
            nn.GELU()
        )
    
    def forward(self, high_feat: torch.Tensor, 
                low_feat: torch.Tensor) -> torch.Tensor:
        """
        Aggregate features from both streams.
        
        Args:
            high_feat: High-res features (B, C_h, H, W)
            low_feat: Low-res features (B, C_l, H/s, W/s)
            
        Returns:
            Aggregated features (B, out_channels, H, W)
        """
        # Upsample low-res to match high-res
        low_up = self.upsample_low(low_feat)
        
        # Concatenate
        concat = torch.cat([high_feat, low_up], dim=1)
        
        # Project to output channels
        output = self.projection(concat)
        
        return output


# =============================================================================
# HRNetV2-Mamba Backbone: Complete Dual-Stream Architecture
# =============================================================================

class HRNetV2MambaBackbone(nn.Module):
    """
    HRNetV2 Backbone with Mamba (SS2D) and Spectral Gating.
    
    Dual-stream architecture maintaining high-resolution features throughout:
    
    Stream 1 (High-Res): H/4 resolution
        - Receives: Energy and Frequency signals (physics features)
        - Purpose: Preserve boundary details
        
    Stream 2 (Low-Res): H/8 resolution
        - Receives: Intensity signal (semantic features)
        - Purpose: Capture global context
    
    The streams are periodically fused via MultiScaleFusion modules.
    Output is aggregated from all resolutions at the highest resolution.
    
    Args:
        in_channels: Input channels (3 for Intensity + Rx + Ry)
        base_channels: Base channel count
        num_stages: Number of processing stages
        blocks_per_stage: Number of SpectralVSSBlocks per stage
        mamba_depth: Depth within each SpectralVSSBlock
        img_size: Input image size (for spectral weight initialization)
    """
    
    def __init__(self, in_channels: int = 3, base_channels: int = 64,
                 num_stages: int = 4, blocks_per_stage: int = 2,
                 mamba_depth: int = 2, img_size: int = 256):
        super().__init__()
        
        self.in_channels = in_channels
        self.base_channels = base_channels
        self.num_stages = num_stages
        self.img_size = img_size
        
        # Initial stem: stride 4 reduction
        self.stem = HRNetStem(in_channels, base_channels, stride=4)
        
        # Initial feature sizes
        high_res_size = img_size // 4  # H/4
        low_res_size = img_size // 8   # H/8
        
        high_channels = base_channels
        low_channels = base_channels * 2
        
        # Create initial low-res stream from high-res
        self.create_low_stream = nn.Sequential(
            nn.Conv2d(high_channels, low_channels, kernel_size=3, stride=2, padding=1, bias=False),
            nn.GroupNorm(min(32, low_channels), low_channels),
            nn.GELU()
        )
        
        # High-resolution stream stages
        self.high_res_stages = nn.ModuleList()
        
        # Low-resolution stream stages
        self.low_res_stages = nn.ModuleList()
        
        # Fusion modules (after each stage)
        self.fusion_modules = nn.ModuleList()
        
        # Build stages
        for stage_idx in range(num_stages):
            # High-res stage
            self.high_res_stages.append(
                HRNetStage(
                    channels=high_channels,
                    height=high_res_size,
                    width=high_res_size,
                    num_blocks=blocks_per_stage,
                    mamba_depth=mamba_depth
                )
            )
            
            # Low-res stage
            self.low_res_stages.append(
                HRNetStage(
                    channels=low_channels,
                    height=low_res_size,
                    width=low_res_size,
                    num_blocks=blocks_per_stage,
                    mamba_depth=mamba_depth
                )
            )
            
            # Fusion module
            self.fusion_modules.append(
                MultiScaleFusion(
                    high_channels=high_channels,
                    low_channels=low_channels,
                    scale_factor=2
                )
            )
        
        # Aggregation layer
        self.aggregation = AggregationLayer(
            high_channels=high_channels,
            low_channels=low_channels,
            out_channels=high_channels + low_channels,
            scale_factor=2
        )
        
        # Store output channels
        self.out_channels = high_channels + low_channels
        self.feature_size = high_res_size
    
    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        """
        Forward pass through dual-stream backbone.
        
        Args:
            x: Input tensor (B, in_channels, H, W)
            
        Returns:
            Dictionary containing:
            - 'features': Aggregated features (B, out_channels, H/4, W/4)
            - 'high_res': High-res stream output (B, C_h, H/4, W/4)
            - 'low_res': Low-res stream output (B, C_l, H/8, W/8)
        """
        # Initial stem
        high = self.stem(x)  # (B, base_channels, H/4, W/4)
        
        # Create low-res stream
        low = self.create_low_stream(high)  # (B, base_channels*2, H/8, W/8)
        
        # Process through stages with fusion
        for stage_idx in range(self.num_stages):
            # Process each stream
            high = self.high_res_stages[stage_idx](high)
            low = self.low_res_stages[stage_idx](low)
            
            # Fuse streams
            high, low = self.fusion_modules[stage_idx](high, low)
        
        # Aggregate for output
        features = self.aggregation(high, low)
        
        return {
            'features': features,
            'high_res': high,
            'low_res': low
        }


# =============================================================================
# Testing
# =============================================================================



### egm_net.py


In [ ]:
"""
Energy-Gated Gabor Mamba Network (EGM-Net).

A hybrid architecture for medical image segmentation combining physics-based
signal processing with deep learning for artifact-free boundary delineation.

Architecture Components:
    1. HRNetV2-Mamba Backbone: Dual-stream with high-res preservation
    2. Monogenic Signal Processing: Physics-based edge detection via Riesz transform
    3. RBF Constellation Head: Gaussian classifier for coarse segmentation
    4. Energy-Gated Gabor Implicit: FiLM-conditioned fine segmentation
    5. Energy-Gated Fusion: Selective boundary refinement

Key Innovations:
    - Dual-path decoding: Coarse (RBF) + Fine (Implicit) branches
    - Energy gating: Suppress artifacts in flat/homogeneous regions
    - Gabor basis: Localized oscillations prevent Gibbs ringing artifacts
    - Vector rotation augmentation: Maintains Riesz consistency under rotation

References:
    [1] Felsberg & Sommer, "The Monogenic Signal," IEEE TSP, 2001.
    [2] Wang et al., "Deep High-Resolution Representation Learning," CVPR 2020.
    [3] Liu et al., "VMamba: Visual State Space Model," arXiv, 2024.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple, Dict
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple, Dict
import math

# Handle imports for both package and standalone usage
try:
except ImportError:
    try:
        # Standalone or different path structure fallback
        import sys
        sys.path.append("..")
    except ImportError:
        # Testing fallback
        pass


# =============================================================================
# Energy-Gated Fusion Module
# =============================================================================

class EnergyGatedFusion(nn.Module):
    """
    Energy-Gated Fusion Module.
    
    Uses monogenic energy map to gate the implicit (fine) branch:
    - High energy (edges): Activate fine branch for sharp boundaries
    - Low energy (flat): Suppress fine branch to avoid artifacts
    
    Final output = Coarse + Energy × (Fine - Coarse)
    """
    
    def __init__(self, temperature: float = 1.0):
        """
        Initialize Energy-Gated Fusion.
        
        Args:
            temperature: Softness of gating (lower = harder gating)
        """
        super().__init__()
        self.temperature = temperature
        self.gate_scale = nn.Parameter(torch.ones(1))
        self.gate_bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, coarse: torch.Tensor, fine: torch.Tensor, 
                energy: torch.Tensor) -> torch.Tensor:
        """
        Fuse coarse and fine predictions using energy gating.
        
        Args:
            coarse: Coarse prediction (B, C, H, W)
            fine: Fine prediction (B, C, H, W) 
            energy: Energy map (B, 1, H, W) in [0, 1]
            
        Returns:
            Fused prediction (B, C, H, W)
        """
        # Resize energy to match prediction size
        if energy.shape[-2:] != coarse.shape[-2:]:
            energy = F.interpolate(energy, size=coarse.shape[-2:], 
                                   mode='bilinear', align_corners=True)
        
        # Resize fine to match coarse if needed
        if fine.shape[-2:] != coarse.shape[-2:]:
            fine = F.interpolate(fine, size=coarse.shape[-2:],
                                mode='bilinear', align_corners=True)
        
        # Apply learnable scaling and temperature
        gate = torch.sigmoid((energy * self.gate_scale + self.gate_bias) / self.temperature)
        
        # Blend: high energy → use fine, low energy → use coarse
        output = coarse + gate * (fine - coarse)
        
        return output


# =============================================================================
# EGM-Net: Main Model
# =============================================================================

class EGMNet(nn.Module):
    """
    Energy-Gated Gabor Mamba Network (EGM-Net).
    
    Full architecture with HRNetV2-Mamba backbone:
    
    1. Input: 3 channels (Intensity, Rx, Ry) or 1 channel (Intensity only)
    2. Backbone: HRNetV2-Mamba (dual-stream, maintains high-res throughout)
    3. Coarse Head: RBF Constellation (Gaussian classifier with PSK prototypes)
    4. Fine Head: Energy-Gated Gabor Implicit (FiLM-conditioned)
    5. Fusion: Energy-gated blending of coarse and fine
    
    The energy map (from Monogenic Signal) gates the fine branch:
    - High energy regions (boundaries): Use fine branch for sharp edges
    - Low energy regions (flat): Use coarse branch to avoid artifacts
    
    Args:
        in_channels: Number of input channels (1 or 3)
        num_classes: Number of segmentation classes
        img_size: Input image size
        base_channels: Base channel count for backbone
        num_stages: Number of backbone stages
        use_hrnet: Use HRNetV2 backbone (True) or simpler encoder (False)
        implicit_hidden: Hidden dim for implicit decoder
        implicit_layers: Number of implicit decoder layers
        num_frequencies: Number of Gabor frequencies
    """
    
    def __init__(self, in_channels: int = 3, num_classes: int = 4,
                 img_size: int = 256, base_channels: int = 64,
                 num_stages: int = 4, use_hrnet: bool = True,
                 implicit_hidden: int = 256, implicit_layers: int = 4,
                 num_frequencies: int = 64):
        super().__init__()
        
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.img_size = img_size
        self.use_hrnet = use_hrnet
        
        # 1. Monogenic Energy Extractor (fixed, non-trainable)
        # For computing energy maps from input images
        self.energy_extractor = EnergyMap(normalize=True, smoothing_sigma=1.0)
        
        # 2. Backbone
        if use_hrnet:
            self.backbone = HRNetV2MambaBackbone(
                in_channels=in_channels,
                base_channels=base_channels,
                num_stages=num_stages,
                blocks_per_stage=2,
                mamba_depth=2,
                img_size=img_size
            )
            backbone_channels = self.backbone.out_channels
        else:
            # Fallback to simpler encoder for testing
            self.backbone = SimpleEncoder(
                in_channels=in_channels,
                base_channels=base_channels,
                num_stages=num_stages
            )
            backbone_channels = base_channels * (2 ** (num_stages - 1))
        
        self.backbone_channels = backbone_channels
        
        # 3. Coarse Head: RBF Constellation
        self.coarse_head = RBFConstellationHead(
            in_channels=backbone_channels,
            num_classes=num_classes,
            embedding_dim=2,
            init_gamma=1.0
        )
        
        # 4. Fine Head: Energy-Gated Gabor Implicit
        self.fine_head = EnergyGatedImplicitHead(
            feature_channels=backbone_channels,
            num_classes=num_classes,
            hidden_dim=implicit_hidden,
            num_layers=implicit_layers,
            num_frequencies=num_frequencies
        )
        
        # 5. Energy-Gated Fusion
        self.fusion = EnergyGatedFusion(temperature=1.0)
        
        # Store feature size for output upsampling
        if use_hrnet:
            self.feature_size = img_size // 4
        else:
            self.feature_size = img_size // (2 ** num_stages)
    
    def _compute_energy(self, x: torch.Tensor) -> Tuple[torch.Tensor, dict]:
        """
        Compute energy map from input.
        
        Args:
            x: Input tensor (B, C, H, W)
            
        Returns:
            Tuple of (energy_map, monogenic_outputs)
        """
        with torch.no_grad():
            # Use first channel (intensity) for energy computation
            x_gray = x[:, 0:1] if x.shape[1] > 1 else x
            energy, mono_out = self.energy_extractor(x_gray)
        return energy, mono_out
    
    def forward(self, x: torch.Tensor, 
                output_size: Optional[Tuple[int, int]] = None,
                return_intermediates: bool = True) -> Dict[str, torch.Tensor]:
        """
        Forward pass.
        
        Args:
            x: Input image (B, C, H, W)
               - If C=3: Expected [Intensity, Rx, Ry]
               - If C=1: Intensity only, energy computed online
            output_size: Optional output resolution (default: input size)
            return_intermediates: Whether to return coarse/fine/energy separately
            
        Returns:
            Dictionary containing:
            - 'output': Final fused segmentation (B, num_classes, H, W)
            - 'coarse': Coarse branch output (B, num_classes, H, W)
            - 'fine': Fine branch output (B, num_classes, H, W)
            - 'energy': Energy map (B, 1, H, W)
            - 'embeddings': 2D constellation embeddings (B, 2, H, W) [optional]
        """
        B, C, H, W = x.shape
        
        if output_size is None:
            output_size = (H, W)
        
        # 1. Compute or extract energy map
        if C >= 3:
            # If Riesz components are provided, compute energy from them
            intensity = x[:, 0:1]  # (B, 1, H, W)
            riesz_x = x[:, 1:2]    # (B, 1, H, W)  
            riesz_y = x[:, 2:3]    # (B, 1, H, W)
            
            # Energy = sqrt(I^2 + Rx^2 + Ry^2)
            energy = torch.sqrt(intensity**2 + riesz_x**2 + riesz_y**2 + 1e-8)
            # Normalize to [0, 1]
            energy = (energy - energy.min()) / (energy.max() - energy.min() + 1e-8)
        else:
            # Compute energy online using Monogenic Signal
            energy, _ = self._compute_energy(x)
        
        # 2. Backbone feature extraction
        if self.use_hrnet:
            backbone_out = self.backbone(x)
            features = backbone_out['features']
        else:
            features = self.backbone(x)
        
        # 3. Coarse Head (RBF Constellation)
        coarse_logits, embeddings = self.coarse_head(features)
        
        # Upsample coarse to output size
        coarse = F.interpolate(coarse_logits, size=output_size,
                              mode='bilinear', align_corners=True)
        
        # 4. Fine Head (Energy-Gated Gabor Implicit)
        # Resize energy to feature size for implicit head
        energy_for_fine = F.interpolate(energy, size=features.shape[-2:],
                                        mode='bilinear', align_corners=True)
        
        fine_logits = self.fine_head(features, energy_for_fine, output_size=output_size)
        
        # Ensure fine is same size as coarse
        if fine_logits.shape[-2:] != output_size:
            fine_logits = F.interpolate(fine_logits, size=output_size,
                                       mode='bilinear', align_corners=True)
        
        # 5. Energy-Gated Fusion
        output = self.fusion(coarse, fine_logits, energy)
        
        if return_intermediates:
            return {
                'output': output,
                'coarse': coarse,
                'fine': fine_logits,
                'energy': energy,
                'embeddings': embeddings
            }
        else:
            return {'output': output}
    
    def inference(self, x: torch.Tensor, 
                  output_size: Optional[Tuple[int, int]] = None) -> torch.Tensor:
        """
        Simplified inference returning only final output.
        
        Args:
            x: Input image (B, C, H, W)
            output_size: Optional output resolution
            
        Returns:
            Segmentation logits (B, num_classes, H, W)
        """
        return self.forward(x, output_size, return_intermediates=False)['output']
    
    def sample_points(self, coarse_logits: torch.Tensor, 
                      energy_map: torch.Tensor, 
                      num_samples: int = 4096) -> torch.Tensor:
        """
        Sample points based on Uncertainty + Energy logic.
        
        Logic:
           1. Compute Uncertainty Map (Entropy or Margin) from coarse predictions.
           2. Combine with Energy Map: P_sample ∝ (Uncertainty + Energy)
           3. Sample coordinates from this distribution.
           
        Args:
            coarse_logits: Coarse segmentation logits (B, num_classes, H, W)
            energy_map: Monogenic energy map (B, 1, H, W)
            num_samples: Number of points to sample per image
            
        Returns:
            Sampled coordinates (B, N, 2) in [-1, 1] range
        """
        B, C, H, W = coarse_logits.shape
        device = coarse_logits.device
        
        # 1. Compute Uncertainty (1 - max_prob)
        # Softmax probabilities
        probs = F.softmax(coarse_logits, dim=1)
        # Max probability per pixel
        max_prob, _ = probs.max(dim=1, keepdim=True)  # (B, 1, H, W)
        # Uncertainty: High where max_prob is low (near 1/num_classes)
        uncertainty = 1.0 - max_prob 
        
        # 2. Resizing Energy Map to match coarse logits
        if energy_map.shape[-2:] != (H, W):
            energy_map = F.interpolate(
                energy_map, size=(H, W), mode='bilinear', align_corners=True
            )
            
        # 3. Combine Uncertainty + Energy for Sampling Probability
        # Normalize both to [0, 1] for balanced combination
        uncertainty = (uncertainty - uncertainty.min()) / (uncertainty.max() - uncertainty.min() + 1e-8)
        # Weighting: Bias strongly towards uncertainty (boundaries) and high energy
        sample_weights = uncertainty + 0.5 * energy_map
        
        # Flatten for sampling
        sample_weights_flat = sample_weights.view(B, -1)  # (B, H*W)
        
        # 4. Multinomial Sampling
        # Sample indices based on weights
        indices = torch.multinomial(sample_weights_flat, num_samples, replacement=True)  # (B, N)
        
        # Convert indices to coordinates [-1, 1]
        # x_idx = index % W, y_idx = index // W
        x_idx = indices % W
        y_idx = indices // W
        
        # Normalize to [-1, 1]
        # (x + 0.5) / W * 2 - 1  (using +0.5 to sample pixel centers)
        x_norm = (x_idx.float() + 0.5) / W * 2.0 - 1.0
        y_norm = (y_idx.float() + 0.5) / H * 2.0 - 1.0
        
        coords = torch.stack([x_norm, y_norm], dim=-1)  # (B, N, 2)
        
        return coords
    
    def query_points(self, x: torch.Tensor, 
                     coords: torch.Tensor) -> torch.Tensor:
        """
        Query segmentation at arbitrary coordinates (implicit representation).
        
        This enables resolution-free inference - zoom into boundaries at any
        resolution without pixelation.
        
        Args:
            x: Input image (B, C, H, W)
            coords: Query coordinates (B, N, 2) normalized to [-1, 1]
            
        Returns:
            Segmentation logits at query points (B, N, num_classes)
        """
        B, C, H, W = x.shape
        
        # Compute energy
        if C >= 3:
            intensity = x[:, 0:1]
            riesz_x = x[:, 1:2]
            riesz_y = x[:, 2:3]
            energy = torch.sqrt(intensity**2 + riesz_x**2 + riesz_y**2 + 1e-8)
            energy = (energy - energy.min()) / (energy.max() - energy.min() + 1e-8)
        else:
            energy, _ = self._compute_energy(x)
        
        # Get backbone features
        if self.use_hrnet:
            backbone_out = self.backbone(x)
            features = backbone_out['features']
        else:
            features = self.backbone(x)
        
        # Sample features and energy at query coordinates
        N = coords.shape[1]
        grid = coords.view(B, 1, N, 2)
        
        # Project features
        features_proj = self.fine_head.feature_proj(features)
        
        # Sample at coordinates
        feat_sampled = F.grid_sample(
            features_proj, grid, mode='bilinear',
            padding_mode='border', align_corners=True
        ).squeeze(2).permute(0, 2, 1)  # (B, N, C)
        
        energy_sampled = F.grid_sample(
            energy, grid, mode='bilinear',
            padding_mode='border', align_corners=True
        ).squeeze(2).permute(0, 2, 1)  # (B, N, 1)
        
        # Implicit decoding with energy gating
        point_logits = self.fine_head.implicit_decoder(coords, feat_sampled, energy_sampled)
        
        return point_logits


# =============================================================================
# Simple Encoder (Fallback for testing)
# =============================================================================

class SimpleEncoder(nn.Module):
    """Simple encoder for testing without full HRNet."""
    
    def __init__(self, in_channels: int = 3, base_channels: int = 64,
                 num_stages: int = 4):
        super().__init__()
        
        layers = []
        channels = in_channels
        out_channels = base_channels
        
        for i in range(num_stages):
            layers.append(nn.Conv2d(channels, out_channels, 3, stride=2, padding=1))
            layers.append(nn.GroupNorm(min(32, out_channels), out_channels))
            layers.append(nn.GELU())
            channels = out_channels
            out_channels = min(out_channels * 2, 512)
        
        self.layers = nn.Sequential(*layers)
        self.out_channels = channels
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)


# =============================================================================
# EGM-Net Lite (Lightweight version)
# =============================================================================

class EGMNetLite(nn.Module):
    """
    Lightweight version of EGM-Net for faster training/inference.
    
    Reduced channels and stages for lower memory/compute.
    """
    
    def __init__(self, in_channels: int = 1, num_classes: int = 2,
                 img_size: int = 256):
        super().__init__()
        
        self.model = EGMNet(
            in_channels=in_channels,
            num_classes=num_classes,
            img_size=img_size,
            base_channels=32,
            num_stages=3,
            use_hrnet=False,  # Use simpler encoder
            implicit_hidden=128,
            implicit_layers=3,
            num_frequencies=32
        )
    
    def forward(self, x, output_size=None):
        return self.model(x, output_size)
    
    def inference(self, x, output_size=None):
        return self.model.inference(x, output_size)


# =============================================================================
# Testing
# =============================================================================



### physics_loss.py


In [ ]:
"""
Physics-Inspired Dual Loss Function
Combines spatial (Dice + CE) and frequency domain losses for medical image segmentation.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional


class DiceLoss(nn.Module):
    """Dice Loss for binary/multi-class segmentation."""
    
    def __init__(self, smooth: float = 1e-5, reduction: str = "mean"):
        """
        Initialize Dice Loss.
        
        Args:
            smooth: Smoothing constant to avoid division by zero
            reduction: 'mean' or 'sum'
        """
        super().__init__()
        self.smooth = smooth
        self.reduction = reduction
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Compute Dice Loss.
        
        Args:
            pred: Prediction logits of shape (B, C, H, W)
            target: Ground truth labels of shape (B, H, W) or (B, C, H, W)
            
        Returns:
            Scalar Dice loss
        """
        # Convert logits to probabilities
        pred = torch.softmax(pred, dim=1)
        
        # Ensure target has same shape as pred for multi-class
        if target.ndim == 3:  # (B, H, W) -> convert to one-hot
            target = F.one_hot(target.long(), num_classes=pred.shape[1])
            target = target.permute(0, 3, 1, 2).float()
        
        # Flatten spatial dimensions
        pred = pred.view(pred.shape[0], pred.shape[1], -1)
        target = target.view(target.shape[0], target.shape[1], -1)
        
        # Compute Dice score
        intersection = torch.sum(pred * target, dim=2)
        union = torch.sum(pred, dim=2) + torch.sum(target, dim=2)
        
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        
        # Return loss (1 - Dice)
        loss = 1.0 - dice
        
        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        else:
            return loss


class FocalLoss(nn.Module):
    """Focal Loss for handling class imbalance."""
    
    def __init__(self, alpha: float = 0.25, gamma: float = 2.0,
                 reduction: str = "mean"):
        """
        Initialize Focal Loss.
        
        Args:
            alpha: Weighting factor for class 1
            gamma: Focusing parameter (higher = more focus on hard examples)
            reduction: 'mean' or 'sum'
        """
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Compute Focal Loss.
        
        Args:
            pred: Prediction logits of shape (B, C, H, W)
            target: Ground truth labels of shape (B, H, W)
            
        Returns:
            Scalar Focal loss
        """
        # Get class probabilities
        p = torch.softmax(pred, dim=1)
        
        # Get class log probabilities
        ce = F.cross_entropy(pred, target.long(), reduction='none')
        
        # Get probability of true class
        p_t = torch.gather(p, 1, target.long().unsqueeze(1)).squeeze(1)
        
        # Compute focal loss
        focal_weight = (1.0 - p_t) ** self.gamma
        focal_loss = focal_weight * ce
        
        if self.reduction == "mean":
            return focal_loss.mean()
        elif self.reduction == "sum":
            return focal_loss.sum()
        else:
            return focal_loss


class FrequencyLoss(nn.Module):
    """
    Frequency Domain Loss for enforcing edge sharpness and detail preservation.
    Computes L2 distance between FFT of prediction and ground truth.
    """
    
    def __init__(self, weight: float = 0.1):
        """
        Initialize FrequencyLoss.
        
        Args:
            weight: Weight factor for this loss component in combined loss
        """
        super().__init__()
        self.weight = weight
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Compute frequency domain loss.
        
        Uses FFT to compare frequency components, emphasizing edge preservation.
        
        Args:
            pred: Prediction tensor of shape (B, C, H, W) or (B, H, W)
            target: Ground truth tensor of same shape as pred
            
        Returns:
            Scalar frequency loss
        """
        # Ensure both have batch and channel dimensions
        if pred.ndim == 3:
            pred = pred.unsqueeze(1)
        if target.ndim == 3:
            target = target.unsqueeze(1)
        
        # Flatten to single channel for FFT comparison
        if pred.shape[1] > 1:
            # For multi-channel, convert to grayscale by averaging
            pred = pred.mean(dim=1, keepdim=True)
        if target.shape[1] > 1:
            target = target.mean(dim=1, keepdim=True)
        
        # Apply FFT to convert to frequency domain
        pred_freq = torch.fft.rfft2(pred, dim=(-2, -1), norm="ortho")
        target_freq = torch.fft.rfft2(target, dim=(-2, -1), norm="ortho")
        
        # Compute L2 distance in frequency domain
        # Consider both magnitude and phase information
        loss_real = F.mse_loss(pred_freq.real, target_freq.real, reduction='mean')
        loss_imag = F.mse_loss(pred_freq.imag, target_freq.imag, reduction='mean')
        
        return loss_real + loss_imag


class SpectralDualLoss(nn.Module):
    """
    Combined Spectral Dual Loss.
    
    Combines:
    1. Spatial losses (Dice + Focal CE) - for overall shape and class balance
    2. Frequency loss - for edge sharpness and boundary preservation
    """
    
    def __init__(self, spatial_weight: float = 1.0, freq_weight: float = 0.1,
                 use_dice: bool = True, use_focal: bool = True):
        """
        Initialize SpectralDualLoss.
        
        Args:
            spatial_weight: Weight for spatial losses
            freq_weight: Weight for frequency loss
            use_dice: Whether to include Dice loss
            use_focal: Whether to include Focal loss (else CrossEntropy)
        """
        super().__init__()
        self.spatial_weight = spatial_weight
        self.freq_weight = freq_weight
        self.use_dice = use_dice
        self.use_focal = use_focal
        
        # Spatial losses
        if use_dice:
            self.dice_loss = DiceLoss(smooth=1e-5)
        
        if use_focal:
            self.focal_loss = FocalLoss(alpha=0.25, gamma=2.0)
        else:
            self.ce_loss = nn.CrossEntropyLoss()
        
        # Frequency loss
        self.freq_loss = FrequencyLoss(weight=freq_weight)
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor,
                return_components: bool = False) -> torch.Tensor:
        """
        Compute combined loss.
        
        Args:
            pred: Prediction logits of shape (B, C, H, W)
            target: Ground truth labels of shape (B, H, W)
            return_components: If True, return dict with individual loss components
            
        Returns:
            Scalar combined loss, or dict if return_components=True
        """
        # Ensure target is on same device as pred
        target = target.to(pred.device)
        
        # Spatial losses
        spatial_loss = 0.0
        losses_dict = {}
        
        if self.use_dice:
            dice = self.dice_loss(pred, target)
            spatial_loss = spatial_loss + dice
            losses_dict['dice'] = dice.item()
        
        if self.use_focal:
            focal = self.focal_loss(pred, target)
            spatial_loss = spatial_loss + focal
            losses_dict['focal'] = focal.item()
        else:
            ce = self.ce_loss(pred, target)
            spatial_loss = spatial_loss + ce
            losses_dict['ce'] = ce.item()
        
        # Frequency loss
        # For frequency loss, we need to extract the predicted class (argmax) and compare
        pred_probs = torch.softmax(pred, dim=1)
        pred_class = torch.argmax(pred_probs, dim=1)  # (B, H, W)
        
        freq = self.freq_loss(pred_class.float(), target.float())
        losses_dict['freq'] = freq.item()
        
        # Weighted combination
        total_loss = (self.spatial_weight * spatial_loss + 
                     self.freq_weight * freq)
        losses_dict['total'] = total_loss.item()
        
        if return_components:
            return total_loss, losses_dict
        else:
            return total_loss


class BoundaryAwareLoss(nn.Module):
    """
    Boundary-aware loss that emphasizes pixels near segmentation boundaries.
    Useful for improving edge precision.
    """
    
    def __init__(self, kernel_size: int = 3, weight: float = 1.0):
        """
        Initialize BoundaryAwareLoss.
        
        Args:
            kernel_size: Size of kernel for computing boundary gradients
            weight: Weight for boundary loss
        """
        super().__init__()
        self.kernel_size = kernel_size
        self.weight = weight
    
    def _compute_boundaries(self, mask: torch.Tensor) -> torch.Tensor:
        """
        Compute boundary mask using gradient.
        
        Args:
            mask: Binary mask of shape (B, H, W)
            
        Returns:
            Boundary map of shape (B, H, W)
        """
        # Convert to float
        mask = mask.float().unsqueeze(1)  # (B, 1, H, W)
        
        # Compute gradients using Sobel-like operation
        kernel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                                dtype=mask.dtype, device=mask.device)
        kernel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
                                dtype=mask.dtype, device=mask.device)
        
        kernel_x = kernel_x.view(1, 1, 3, 3)
        kernel_y = kernel_y.view(1, 1, 3, 3)
        
        grad_x = F.conv2d(mask, kernel_x, padding=1)
        grad_y = F.conv2d(mask, kernel_y, padding=1)
        
        # Compute magnitude of gradient
        grad_magnitude = torch.sqrt(grad_x ** 2 + grad_y ** 2 + 1e-8)
        
        # Threshold to get boundary pixels
        boundary = (grad_magnitude > 0).float().squeeze(1)
        
        return boundary
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        Compute boundary-aware loss.
        
        Args:
            pred: Prediction logits of shape (B, C, H, W)
            target: Ground truth labels of shape (B, H, W)
            
        Returns:
            Scalar loss emphasizing boundaries
        """
        # Get predicted class
        pred_probs = torch.softmax(pred, dim=1)
        pred_class = torch.argmax(pred_probs, dim=1)  # (B, H, W)
        
        # Compute boundary maps
        pred_boundary = self._compute_boundaries(pred_class)
        target_boundary = self._compute_boundaries(target)
        
        # Compute cross-entropy loss weighted by boundary
        ce_loss = F.cross_entropy(pred, target.long(), reduction='none')
        
        # Apply boundary weight (higher loss for boundary pixels)
        boundary_weight = (pred_boundary + target_boundary).clamp(0, 1)
        boundary_weight = 1.0 + boundary_weight  # Weight between 1 and 2
        
        weighted_loss = ce_loss * boundary_weight
        
        return weighted_loss.mean()


class EyeOpeningLoss(nn.Module):
    """
    Eye Opening Loss (Entropy Minimization at Boundaries).
    
    Based on the Eye Diagram concept from telecommunications:
    - Clear "eye" opening = decisive predictions (0 or 1)
    - Closed "eye" = uncertain predictions (~0.5)
    
    This loss penalizes predictions near 0.5 at boundary regions,
    forcing the model to make decisive (sharp) boundary predictions.
    
    L_eye = 4 × p × (1 - p)
    
    This reaches maximum (1.0) when p=0.5 and minimum (0.0) when p=0 or p=1.
    
    Features:
    - Warm-up scheduling: Only activates after N epochs
    - Annealing: Weight gradually increases to prevent early destabilization
    - Boundary focus: Optionally weight by energy/boundary map
    
    Args:
        warmup_epochs: Number of epochs before activating this loss
        max_weight: Maximum weight for this loss component
        anneal_rate: Rate of weight increase per epoch after warm-up
        smooth: Smoothing constant for numerical stability
    """
    
    def __init__(self, warmup_epochs: int = 5, max_weight: float = 0.1,
                 anneal_rate: float = 0.02, smooth: float = 1e-7):
        super().__init__()
        self.warmup_epochs = warmup_epochs
        self.max_weight = max_weight
        self.anneal_rate = anneal_rate
        self.smooth = smooth
    
    def get_weight(self, epoch: int) -> float:
        """
        Compute loss weight based on current epoch.
        
        Args:
            epoch: Current training epoch (0-indexed)
            
        Returns:
            Loss weight (0 during warm-up, then annealed up to max_weight)
        """
        if epoch < self.warmup_epochs:
            return 0.0
        return min(self.max_weight, self.anneal_rate * (epoch - self.warmup_epochs))
    
    def forward(self, logits: torch.Tensor, 
                energy_map: Optional[torch.Tensor] = None,
                epoch: int = 0) -> torch.Tensor:
        """
        Compute Eye Opening Loss.
        
        Args:
            logits: Prediction logits of shape (B, C, H, W) or (B, N, C)
            energy_map: Optional energy map (B, 1, H, W) to focus on boundaries
            epoch: Current training epoch for weight scheduling
            
        Returns:
            Scalar eye opening loss
        """
        # Get weight for current epoch
        weight = self.get_weight(epoch)
        
        if weight <= 0:
            return torch.tensor(0.0, device=logits.device, requires_grad=False)
        
        # Convert logits to probabilities
        if logits.dim() == 4:
            # (B, C, H, W) -> softmax over classes
            probs = torch.softmax(logits, dim=1)
            # For multi-class, we want high confidence for ANY class
            # Maximum probability per pixel (how confident the prediction is)
            max_probs = probs.max(dim=1, keepdim=True)[0]  # (B, 1, H, W)
        else:
            # (B, N, C) -> softmax over classes
            probs = torch.softmax(logits, dim=-1)
            max_probs = probs.max(dim=-1, keepdim=True)[0]  # (B, N, 1)
        
        # Eye opening: penalize uncertainty (probs near 0.5)
        # L = 4 × p × (1 - p), maximizes at p=0.5
        eye_loss = 4 * max_probs * (1 - max_probs)
        
        # Apply energy weighting (focus on boundaries) if provided
        if energy_map is not None:
            if energy_map.shape[-2:] != eye_loss.shape[-2:]:
                energy_map = F.interpolate(
                    energy_map, size=eye_loss.shape[-2:],
                    mode='bilinear', align_corners=True
                )
            eye_loss = eye_loss * energy_map
        
        # Return weighted mean
        return weight * eye_loss.mean()


class EGMCombinedLoss(nn.Module):
    """
    Combined Loss for EGM-Net Training.
    
    Combines:
    1. Coarse Loss: Dice + CE for the coarse segmentation
    2. Fine Loss: BCE/CE for point-sampled fine predictions
    3. Eye Opening Loss: Entropy minimization at boundaries (with warm-up)
    4. Consistency Loss: Agreement between coarse and fine branches
    
    Args:
        dice_weight: Weight for Dice loss
        ce_weight: Weight for CrossEntropy loss
        fine_weight: Weight for fine branch loss
        eye_weight: Max weight for Eye Opening loss
        consistency_weight: Weight for coarse-fine consistency
        eye_warmup: Epochs before activating Eye Opening loss
    """
    
    def __init__(self, dice_weight: float = 1.0, ce_weight: float = 1.0,
                 fine_weight: float = 1.0, eye_weight: float = 0.1,
                 consistency_weight: float = 0.1, eye_warmup: int = 5):
        super().__init__()
        
        self.dice_weight = dice_weight
        self.ce_weight = ce_weight
        self.fine_weight = fine_weight
        self.eye_weight = eye_weight
        self.consistency_weight = consistency_weight
        
        # Loss components
        self.dice_loss = DiceLoss()
        self.ce_loss = nn.CrossEntropyLoss()
        self.eye_loss = EyeOpeningLoss(
            warmup_epochs=eye_warmup, 
            max_weight=eye_weight,
            anneal_rate=0.02
        )
    
    def forward(self, outputs: dict, target: torch.Tensor,
                point_logits: Optional[torch.Tensor] = None,
                point_labels: Optional[torch.Tensor] = None,
                epoch: int = 0) -> dict:
        """
        Compute combined loss.
        
        Args:
            outputs: Model outputs dict with 'coarse', 'fine', 'energy'
            target: Ground truth mask (B, H, W)
            point_logits: Optional point predictions (B, N, C)
            point_labels: Optional point labels (B, N)
            epoch: Current training epoch
            
        Returns:
            Dict with 'total' loss and individual components
        """
        losses = {}
        
        # 1. Coarse branch losses
        coarse = outputs.get('coarse', outputs.get('output'))
        
        # Resize target if needed
        if coarse.shape[-2:] != target.shape[-2:]:
            target_resized = F.interpolate(
                target.unsqueeze(1).float(),
                size=coarse.shape[-2:],
                mode='nearest'
            ).squeeze(1).long()
        else:
            target_resized = target.long()
        
        dice = self.dice_loss(coarse, target_resized)
        ce = self.ce_loss(coarse, target_resized)
        
        losses['dice'] = dice
        losses['ce'] = ce
        
        coarse_loss = self.dice_weight * dice + self.ce_weight * ce
        
        # 2. Fine branch loss (point-based if available)
        fine_loss = torch.tensor(0.0, device=coarse.device)
        if point_logits is not None and point_labels is not None:
            # Point-based cross-entropy
            B, N, C = point_logits.shape
            point_logits_flat = point_logits.view(B * N, C)
            point_labels_flat = point_labels.view(B * N)
            fine_loss = F.cross_entropy(point_logits_flat, point_labels_flat)
            losses['fine'] = fine_loss
        elif 'fine' in outputs:
            # Use spatial fine output
            fine = outputs['fine']
            if fine.shape[-2:] != target_resized.shape[-2:]:
                target_for_fine = F.interpolate(
                    target_resized.unsqueeze(1).float(),
                    size=fine.shape[-2:],
                    mode='nearest'
                ).squeeze(1).long()
            else:
                target_for_fine = target_resized
            fine_loss = self.ce_loss(fine, target_for_fine)
            losses['fine'] = fine_loss
        
        # 3. Eye Opening Loss (with warm-up)
        energy_map = outputs.get('energy', None)
        if 'fine' in outputs:
            eye = self.eye_loss(outputs['fine'], energy_map, epoch)
        else:
            eye = self.eye_loss(coarse, energy_map, epoch)
        losses['eye'] = eye
        
        # 4. Consistency Loss (coarse and fine should agree)
        consistency_loss = torch.tensor(0.0, device=coarse.device)
        if 'fine' in outputs and self.consistency_weight > 0:
            fine = outputs['fine']
            if fine.shape[-2:] != coarse.shape[-2:]:
                fine_resized = F.interpolate(
                    fine, size=coarse.shape[-2:],
                    mode='bilinear', align_corners=True
                )
            else:
                fine_resized = fine
            
            # KL divergence for consistency
            coarse_probs = F.log_softmax(coarse, dim=1)
            fine_probs = F.softmax(fine_resized, dim=1)
            consistency_loss = F.kl_div(coarse_probs, fine_probs, reduction='batchmean')
            losses['consistency'] = consistency_loss
        
        # Total loss
        total = (coarse_loss + 
                 self.fine_weight * fine_loss + 
                 eye + 
                 self.consistency_weight * consistency_loss)
        
        losses['total'] = total
        
        return losses



    # Test losses
    batch_size, num_classes, height, width = 2, 3, 64, 64
    
    # Create dummy predictions and targets
    pred = torch.randn(batch_size, num_classes, height, width)
    target = torch.randint(0, num_classes, (batch_size, height, width))
    
    # Test SpectralDualLoss
    loss_fn = SpectralDualLoss(spatial_weight=1.0, freq_weight=0.1)
    loss, components = loss_fn(pred, target, return_components=True)
    
    print(f"Total Loss: {loss.item():.4f}")
    for name, value in components.items():
        print(f"  {name}: {value:.4f}")
    
    # Test BoundaryAwareLoss
    boundary_loss_fn = BoundaryAwareLoss()
    boundary_loss = boundary_loss_fn(pred, target)
    print(f"\nBoundary Loss: {boundary_loss.item():.4f}")


### 2.1 Mamba Block

### 2.2 Spectral Layers

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional

class SpectralGating(nn.Module):

    def __init__(self, channels: int, height: int, width: int, 
                 threshold: float = 0.1, complex_init: str = "kaiming"):
        
        super().__init__()
        self.channels = channels
        self.height = height
        self.width = width
        self.threshold = threshold
        
        # Create learnable complex weights for frequency domain
        # Shape: (channels, height, width//2 + 1) for rfft2
        self.register_buffer(
            "freq_shape",
            torch.tensor([channels, height, width // 2 + 1], dtype=torch.long)
        )
        
        # Real and Imaginary parts of complex weights
        self.weight_real = nn.Parameter(
            torch.zeros(channels, height, width // 2 + 1)
        )
        self.weight_imag = nn.Parameter(
            torch.zeros(channels, height, width // 2 + 1)
        )
        
        self._init_weights(complex_init)
        
    def _init_weights(self, strategy: str = "kaiming"):
        
        if strategy == "identity":
            # Initialize close to identity (magnitude ~1, phase ~0)
            nn.init.ones_(self.weight_real)
            nn.init.zeros_(self.weight_imag)
        elif strategy == "kaiming":
            # Kaiming initialization adapted for complex numbers
            fan_in = self.height * (self.width // 2 + 1)
            std = (2.0 / fan_in) ** 0.5
            nn.init.normal_(self.weight_real, 0, std)
            nn.init.normal_(self.weight_imag, 0, std)
        else:
            raise ValueError(f"Unknown init strategy: {strategy}")
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        B, C, H, W = x.shape
        
        # Apply FFT to convert to frequency domain
        # rfft2 returns complex tensor
        x_freq = torch.fft.rfft2(x, dim=(-2, -1), norm="ortho")
        
        # Create complex weight matrix: weight_real + 1j * weight_imag
        # Reshape to (1, C, H, W//2+1) for broadcasting
        complex_weight = (
            self.weight_real.unsqueeze(0) + 
            1j * self.weight_imag.unsqueeze(0)
        )
        
        # Apply channel-wise multiplication in frequency domain
        # Shape: (B, C, H, W//2+1) * (1, C, H, W//2+1) -> (B, C, H, W//2+1)
        x_filtered = x_freq * complex_weight
        
        # Optional: Hard thresholding to remove low-amplitude noise
        if self.threshold > 0:
            magnitude = torch.abs(x_filtered)
            mask = magnitude > self.threshold
            x_filtered = x_filtered * mask.float()
        
        # Apply inverse FFT to return to spatial domain
        output = torch.fft.irfft2(x_filtered, s=(H, W), dim=(-2, -1), norm="ortho")
        
        return output

class FrequencyLoss(nn.Module):

    def __init__(self, weight: float = 0.1):
        
        super().__init__()
        self.weight = weight
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        
        # Apply FFT
        pred_freq = torch.fft.rfft2(pred, dim=(-2, -1), norm="ortho")
        target_freq = torch.fft.rfft2(target, dim=(-2, -1), norm="ortho")
        
        # Compute L2 distance in frequency domain
        # Using both magnitude and phase information
        loss_real = F.mse_loss(pred_freq.real, target_freq.real)
        loss_imag = F.mse_loss(pred_freq.imag, target_freq.imag)
        
        return loss_real + loss_imag

if __name__ == "__main__":
    # Test SpectralGating
    batch_size, channels, height, width = 2, 64, 64, 64
    x = torch.randn(batch_size, channels, height, width)
    
    spec_gate = SpectralGating(channels, height, width)
    output = spec_gate(x)
    
    print(f"Input shape: {x.shape}")
    print(f"Output shape: {output.shape}")
    print(f"Module parameters: {sum(p.numel() for p in spec_gate.parameters())}")


### 2.3 Monogenic Signal Processing

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Tuple, Optional

class RieszTransform(nn.Module):

    def __init__(self, epsilon: float = 1e-8):
        
        super().__init__()
        self.epsilon = epsilon
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        
        B, C, H, W = x.shape
        
        # Create frequency grid
        freq_y = torch.fft.fftfreq(H, device=x.device, dtype=x.dtype)
        freq_x = torch.fft.fftfreq(W, device=x.device, dtype=x.dtype)
        freq_y, freq_x = torch.meshgrid(freq_y, freq_x, indexing='ij')
        
        # Compute radial frequency (avoid division by zero)
        radius = torch.sqrt(freq_x**2 + freq_y**2 + self.epsilon)
        
        # Riesz kernels in frequency domain
        # H1 = -j * u / |w|, H2 = -j * v / |w|
        kernel_x = freq_x / radius
        kernel_y = freq_y / radius
        
        # Set DC component to zero
        kernel_x[0, 0] = 0
        kernel_y[0, 0] = 0
        
        # Apply FFT to input
        x_fft = torch.fft.fft2(x)
        
        # Apply Riesz kernels (multiplication by -j in frequency = Hilbert-like)
        # -j * X = real(X) * (-j) + imag(X) * (-j) * j = imag(X) - j*real(X)
        riesz_x_fft = -1j * x_fft * kernel_x.unsqueeze(0).unsqueeze(0)
        riesz_y_fft = -1j * x_fft * kernel_y.unsqueeze(0).unsqueeze(0)
        
        # Inverse FFT
        riesz_x = torch.fft.ifft2(riesz_x_fft).real
        riesz_y = torch.fft.ifft2(riesz_y_fft).real
        
        return riesz_x, riesz_y

class LogGaborFilter(nn.Module):

    def __init__(self, num_scales: int = 4, num_orientations: int = 6,
                 min_wavelength: float = 3.0, mult: float = 2.1,
                 sigma_on_f: float = 0.55):
        
        super().__init__()
        self.num_scales = num_scales
        self.num_orientations = num_orientations
        self.min_wavelength = min_wavelength
        self.mult = mult
        self.sigma_on_f = sigma_on_f
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        B, C, H, W = x.shape
        device = x.device
        dtype = x.dtype
        
        # Create frequency grid
        freq_y = torch.fft.fftfreq(H, device=device, dtype=dtype)
        freq_x = torch.fft.fftfreq(W, device=device, dtype=dtype)
        freq_y, freq_x = torch.meshgrid(freq_y, freq_x, indexing='ij')
        
        # Polar coordinates
        radius = torch.sqrt(freq_x**2 + freq_y**2)
        radius[0, 0] = 1  # Avoid log(0)
        theta = torch.atan2(freq_y, freq_x)
        
        # FFT of input
        x_fft = torch.fft.fft2(x)
        
        outputs = []
        
        for scale in range(self.num_scales):
            wavelength = self.min_wavelength * (self.mult ** scale)
            fo = 1.0 / wavelength  # Center frequency
            
            # Log-Gabor radial component
            log_gabor_radial = torch.exp(
                -(torch.log(radius / fo) ** 2) / (2 * math.log(self.sigma_on_f) ** 2)
            )
            log_gabor_radial[0, 0] = 0  # Zero DC
            
            for orient in range(self.num_orientations):
                angle = orient * math.pi / self.num_orientations
                
                # Angular component
                ds = torch.sin(theta - angle)
                dc = torch.cos(theta - angle)
                dtheta = torch.abs(torch.atan2(ds, dc))
                
                # Angular spread
                angular_spread = torch.exp(
                    -(dtheta ** 2) / (2 * (math.pi / self.num_orientations) ** 2)
                )
                
                # Combined filter
                log_gabor = log_gabor_radial * angular_spread
                
                # Apply filter
                filtered = torch.fft.ifft2(x_fft * log_gabor.unsqueeze(0).unsqueeze(0))
                outputs.append(filtered.abs())
        
        return torch.cat(outputs, dim=1)

class MonogenicSignal(nn.Module):

    def __init__(self, epsilon: float = 1e-8):
        
        super().__init__()
        self.riesz = RieszTransform(epsilon=epsilon)
        self.epsilon = epsilon
    
    def forward(self, x: torch.Tensor) -> dict:
        
        # Get Riesz components
        riesz_x, riesz_y = self.riesz(x)
        
        # Compute amplitude (local energy)
        # A = sqrt(f^2 + h1^2 + h2^2)
        amplitude = torch.sqrt(x**2 + riesz_x**2 + riesz_y**2 + self.epsilon)
        
        # Compute orientation
        # theta = atan2(h2, h1)
        orientation = torch.atan2(riesz_y, riesz_x + self.epsilon)
        
        # Compute phase
        # phi = atan2(sqrt(h1^2 + h2^2), f)
        riesz_magnitude = torch.sqrt(riesz_x**2 + riesz_y**2 + self.epsilon)
        phase = torch.atan2(riesz_magnitude, x + self.epsilon)
        
        return {
            'amplitude': amplitude,
            'phase': phase,
            'orientation': orientation,
            'riesz_x': riesz_x,
            'riesz_y': riesz_y
        }

class EnergyMap(nn.Module):

    def __init__(self, normalize: bool = True, smoothing_sigma: float = 1.0):
        
        super().__init__()
        self.monogenic = MonogenicSignal()
        self.normalize = normalize
        self.smoothing_sigma = smoothing_sigma
        
        # Create Gaussian smoothing kernel
        if smoothing_sigma > 0:
            kernel_size = int(6 * smoothing_sigma) | 1  # Ensure odd
            self.register_buffer('smooth_kernel', self._create_gaussian_kernel(
                kernel_size, smoothing_sigma
            ))
        else:
            self.smooth_kernel = None
    
    def _create_gaussian_kernel(self, kernel_size: int, sigma: float) -> torch.Tensor:
        
        x = torch.arange(kernel_size) - kernel_size // 2
        x = x.float()
        gaussian_1d = torch.exp(-x**2 / (2 * sigma**2))
        gaussian_2d = gaussian_1d.unsqueeze(0) * gaussian_1d.unsqueeze(1)
        gaussian_2d = gaussian_2d / gaussian_2d.sum()
        return gaussian_2d.unsqueeze(0).unsqueeze(0)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, dict]:
        
        # Convert to grayscale if needed
        if x.shape[1] > 1:
            x = x.mean(dim=1, keepdim=True)
        
        # Get monogenic decomposition
        mono_out = self.monogenic(x)
        
        # Energy is the amplitude
        energy = mono_out['amplitude']
        
        # Optional smoothing
        if self.smooth_kernel is not None:
            pad = self.smooth_kernel.shape[-1] // 2
            energy = F.conv2d(energy, self.smooth_kernel, padding=pad)
        
        # Normalize to [0, 1]
        if self.normalize:
            B = energy.shape[0]
            energy_flat = energy.view(B, -1)
            energy_min = energy_flat.min(dim=1, keepdim=True)[0].view(B, 3, 1, 1)
            energy_max = energy_flat.max(dim=1, keepdim=True)[0].view(B, 3, 1, 1)
            energy = (energy - energy_min) / (energy_max - energy_min + 1e-8)
        
        return energy, mono_out

class BoundaryDetector(nn.Module):

    def __init__(self, num_scales: int = 4, num_orientations: int = 6,
                 noise_threshold: float = 0.1):
        
        super().__init__()
        self.log_gabor = LogGaborFilter(num_scales, num_orientations)
        self.num_scales = num_scales
        self.num_orientations = num_orientations
        self.noise_threshold = noise_threshold
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        # Get multi-scale responses
        responses = self.log_gabor(x)  # (B, S*O, H, W)
        
        # Sum across orientations to get edge strength per scale
        B, _, H, W = responses.shape
        responses = responses.view(B, self.num_scales, self.num_orientations, H, W)
        
        # Max across orientations (strongest edge direction)
        edge_strength = responses.max(dim=2)[0]  # (B, S, H, W)
        
        # Sum across scales
        edge_strength = edge_strength.sum(dim=1, keepdim=True)  # (B, 3, H, W)
        
        # Normalize and threshold
        edge_max = edge_strength.view(B, -1).max(dim=1)[0].view(B, 3, 1, 1)
        edge_strength = edge_strength / (edge_max + 1e-8)
        edge_strength = torch.clamp(edge_strength - self.noise_threshold, min=0)
        edge_strength = edge_strength / (1 - self.noise_threshold + 1e-8)
        
        return edge_strength

if __name__ == "__main__":
    # Test Monogenic Signal processing
    print("Testing Monogenic Signal Processing...")
    
    # Create test image with edges
    H, W = 128, 128
    x = torch.zeros(1, 3, H, W)
    x[:, :, 32:96, 32:96] = 1.0  # Square
    
    # Add some noise
    x = x + 0.1 * torch.randn_like(x)
    
    # Test Energy Map
    energy_extractor = EnergyMap(normalize=True)
    energy, mono = energy_extractor(x)
    
    print(f"Input shape: {x.shape}")
    print(f"Energy map shape: {energy.shape}")
    print(f"Energy range: [{energy.min():.3f}, {energy.max():.3f}]")
    print(f"Monogenic components: {list(mono.keys())}")
    
    # Test Boundary Detector
    boundary_detector = BoundaryDetector()
    boundaries = boundary_detector(x)
    
    print(f"Boundary map shape: {boundaries.shape}")
    print(f"Boundary range: [{boundaries.min():.3f}, {boundaries.max():.3f}]")
    
    print("\n✓ All tests passed!")



### 2.4 Gabor Implicit Layers

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Optional, Tuple, List

class GaborBasis(nn.Module):

    def __init__(self, input_dim: int = 2, num_frequencies: int = 64,
                 sigma_range: Tuple[float, float] = (0.1, 2.0),
                 freq_range: Tuple[float, float] = (1.0, 10.0),
                 learnable: bool = True):
        
        super().__init__()
        self.input_dim = input_dim
        self.num_frequencies = num_frequencies
        self.output_dim = num_frequencies * 2  # sin and cos components
        
        # Initialize frequencies uniformly in log space
        log_freqs = torch.linspace(
            math.log(freq_range[0]), 
            math.log(freq_range[1]), 
            num_frequencies
        )
        freqs = torch.exp(log_freqs)
        
        # Initialize sigmas (Gaussian envelope widths)
        sigmas = torch.linspace(sigma_range[0], sigma_range[1], num_frequencies)
        
        # Random orientations for 2D
        orientations = torch.rand(num_frequencies) * 2 * math.pi
        
        # Random phases
        phases = torch.rand(num_frequencies) * 2 * math.pi
        
        # Create direction vectors from orientations
        directions = torch.stack([
            torch.cos(orientations),
            torch.sin(orientations)
        ], dim=-1)  # (num_freq, 2)
        
        if learnable:
            self.freqs = nn.Parameter(freqs)
            self.sigmas = nn.Parameter(sigmas)
            self.directions = nn.Parameter(directions)
            self.phases = nn.Parameter(phases)
        else:
            self.register_buffer('freqs', freqs)
            self.register_buffer('sigmas', sigmas)
            self.register_buffer('directions', directions)
            self.register_buffer('phases', phases)
    
    def forward(self, coords: torch.Tensor) -> torch.Tensor:
        
        # Normalize directions
        directions = F.normalize(self.directions, dim=-1)  # (num_freq, 2)
        
        # Project coordinates onto directions
        # coords: (..., 2), directions: (num_freq, 2)
        proj = torch.matmul(coords, directions.T)  # (..., num_freq)
        
        # Compute Gaussian envelope
        # exp(-proj² / (2σ²))
        sigmas = torch.abs(self.sigmas) + 0.01  # Ensure positive
        gaussian = torch.exp(-proj**2 / (2 * sigmas**2 + 1e-8))
        
        # Compute oscillatory component
        # cos(2πf·proj + φ), sin(2πf·proj + φ)
        freqs = torch.abs(self.freqs) + 0.1  # Ensure positive
        arg = 2 * math.pi * freqs * proj + self.phases
        
        cos_comp = gaussian * torch.cos(arg)
        sin_comp = gaussian * torch.sin(arg)
        
        # Concatenate sin and cos
        gabor_features = torch.cat([cos_comp, sin_comp], dim=-1)
        
        return gabor_features

class FourierFeatures(nn.Module):

    def __init__(self, input_dim: int = 2, num_frequencies: int = 64,
                 scale: float = 10.0, learnable: bool = False):
        
        super().__init__()
        self.input_dim = input_dim
        self.num_frequencies = num_frequencies
        self.output_dim = num_frequencies * 2
        
        # Random frequency matrix
        B = torch.randn(input_dim, num_frequencies) * scale
        
        if learnable:
            self.B = nn.Parameter(B)
        else:
            self.register_buffer('B', B)
    
    def forward(self, coords: torch.Tensor) -> torch.Tensor:
        
        # Project: coords @ B
        proj = 2 * math.pi * torch.matmul(coords, self.B)  # (..., num_freq)
        
        # Sin and cos
        return torch.cat([torch.cos(proj), torch.sin(proj)], dim=-1)

class SIRENLayer(nn.Module):

    def __init__(self, in_features: int, out_features: int, 
                 omega_0: float = 30.0, is_first: bool = False):
        
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.in_features = in_features
        
        self.linear = nn.Linear(in_features, out_features)
        self._init_weights()
    
    def _init_weights(self):
        
        with torch.no_grad():
            if self.is_first:
                # First layer: uniform in [-1/n, 1/n]
                self.linear.weight.uniform_(-1 / self.in_features, 
                                            1 / self.in_features)
            else:
                # Other layers: uniform in [-sqrt(6/n)/ω₀, sqrt(6/n)/ω₀]
                bound = math.sqrt(6 / self.in_features) / self.omega_0
                self.linear.weight.uniform_(-bound, bound)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.sin(self.omega_0 * self.linear(x))

class GaborNet(nn.Module):

    def __init__(self, coord_dim: int = 2, feature_dim: int = 256,
                 hidden_dim: int = 256, output_dim: int = 1,
                 num_layers: int = 4, num_frequencies: int = 64,
                 use_gabor: bool = True, omega_0: float = 30.0):
        
        super().__init__()
        
        # Coordinate encoding
        if use_gabor:
            self.coord_encoder = GaborBasis(
                input_dim=coord_dim,
                num_frequencies=num_frequencies,
                learnable=True
            )
        else:
            self.coord_encoder = FourierFeatures(
                input_dim=coord_dim,
                num_frequencies=num_frequencies,
                learnable=False
            )
        
        coord_encoded_dim = self.coord_encoder.output_dim
        
        # Input dimension: encoded coords + features
        input_dim = coord_encoded_dim + feature_dim
        
        # Build SIREN network
        layers = []
        
        # First layer
        layers.append(SIRENLayer(input_dim, hidden_dim, omega_0, is_first=True))
        
        # Hidden layers
        for _ in range(num_layers - 2):
            layers.append(SIRENLayer(hidden_dim, hidden_dim, omega_0, is_first=False))
        
        # Final layer (linear, no sine activation)
        layers.append(nn.Linear(hidden_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, coords: torch.Tensor, features: torch.Tensor) -> torch.Tensor:
        
        # Encode coordinates
        coord_encoded = self.coord_encoder(coords)  # (B, N, coord_encoded_dim)
        
        # Concatenate with features
        x = torch.cat([coord_encoded, features], dim=-1)  # (B, N, input_dim)
        
        # Pass through network
        output = self.network(x)  # (B, N, output_dim)
        
        return output

class ImplicitSegmentationHead(nn.Module):

    def __init__(self, feature_channels: int = 64, num_classes: int = 2,
                 hidden_dim: int = 256, num_layers: int = 4,
                 num_frequencies: int = 64, use_gabor: bool = True):
        
        super().__init__()
        
        self.feature_channels = feature_channels
        self.num_classes = num_classes
        
        # Feature projector (reduce channel dimension)
        self.feature_proj = nn.Sequential(
            nn.Conv2d(feature_channels, hidden_dim, kernel_size=1),
            nn.GroupNorm(32, hidden_dim),
            nn.GELU()
        )
        
        # Implicit decoder
        self.implicit_decoder = GaborNet(
            coord_dim=2,
            feature_dim=hidden_dim,
            hidden_dim=hidden_dim,
            output_dim=num_classes,
            num_layers=num_layers,
            num_frequencies=num_frequencies,
            use_gabor=use_gabor
        )
    
    def sample_features(self, feature_map: torch.Tensor, 
                       coords: torch.Tensor) -> torch.Tensor:
        
        B, C, H, W = feature_map.shape
        N = coords.shape[1]
        
        # Reshape coords for grid_sample: (B, N, 3, 2) -> (B, 3, N, 2)
        # grid_sample expects (B, H, W, 2) where last dim is (x, y)
        grid = coords.view(B, 3, N, 2)
        
        # Sample using bilinear interpolation
        # feature_map: (B, C, H, W), grid: (B, 3, N, 2)
        # output: (B, C, 3, N)
        sampled = F.grid_sample(
            feature_map, grid,
            mode='bilinear',
            padding_mode='border',
            align_corners=True
        )
        
        # Reshape: (B, C, 3, N) -> (B, N, C)
        sampled = sampled.squeeze(2).permute(0, 2, 1)
        
        return sampled
    
    def forward(self, feature_map: torch.Tensor, 
                coords: Optional[torch.Tensor] = None,
                output_size: Optional[Tuple[int, int]] = None) -> torch.Tensor:
        
        B, C, H_feat, W_feat = feature_map.shape
        device = feature_map.device
        
        # Project features
        feature_map = self.feature_proj(feature_map)  # (B, hidden_dim, H, W)
        
        # Generate coordinates if not provided
        if coords is None:
            if output_size is None:
                output_size = (H_feat * 4, W_feat * 4)
            
            H_out, W_out = output_size
            
            # Create normalized coordinate grid [-1, 1]
            y = torch.linspace(-1, 3, H_out, device=device)
            x = torch.linspace(-1, 3, W_out, device=device)
            yy, xx = torch.meshgrid(y, x, indexing='ij')
            coords = torch.stack([xx, yy], dim=-1)  # (H_out, W_out, 2)
            coords = coords.view(1, -1, 2).expand(B, -1, -1)  # (B, H*W, 2)
            
            reshape_output = True
        else:
            reshape_output = False
            H_out, W_out = None, None
        
        # Sample features at coordinates
        features = self.sample_features(feature_map, coords)  # (B, N, hidden_dim)
        
        # Implicit decoding
        logits = self.implicit_decoder(coords, features)  # (B, N, num_classes)
        
        # Reshape to image if using grid
        if reshape_output:
            logits = logits.view(B, H_out, W_out, self.num_classes)
            logits = logits.permute(0, 3, 3, 2)  # (B, C, H, W)
        
        return logits

if __name__ == "__main__":
    print("Testing Gabor Implicit Modules...")
    
    # Test Gabor Basis
    print("\n[1] Testing GaborBasis...")
    gabor = GaborBasis(input_dim=2, num_frequencies=32)
    coords = torch.randn(4, 100, 2)  # (B, N, 2)
    encoded = gabor(coords)
    print(f"Input coords: {coords.shape}")
    print(f"Gabor encoded: {encoded.shape}")
    
    # Test GaborNet
    print("\n[2] Testing GaborNet...")
    net = GaborNet(coord_dim=2, feature_dim=64, hidden_dim=128, 
                   output_dim=3, num_layers=3, num_frequencies=32)
    features = torch.randn(4, 100, 64)
    output = net(coords, features)
    print(f"GaborNet output: {output.shape}")
    
    # Test ImplicitSegmentationHead
    print("\n[3] Testing ImplicitSegmentationHead...")
    head = ImplicitSegmentationHead(
        feature_channels=64, num_classes=3,
        hidden_dim=128, num_layers=3, num_frequencies=32
    )
    feature_map = torch.randn(2, 64, 32, 32)
    
    # Test with automatic grid
    seg_output = head(feature_map, output_size=(128, 128))
    print(f"Feature map: {feature_map.shape}")
    print(f"Segmentation output (grid): {seg_output.shape}")
    
    # Test with custom coordinates
    custom_coords = torch.rand(2, 500, 2) * 2 - 1  # Random points in [-1, 1]
    seg_points = head(feature_map, coords=custom_coords)
    print(f"Segmentation output (points): {seg_points.shape}")
    
    print("\n✓ All tests passed!")



### 2.5 Spectral Mamba

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, List

class SpectralVSSBlock(nn.Module):

    def __init__(self, channels: int, height: int, width: int,
                 depth: int = 2, expansion_ratio: float = 2.0,
                 threshold: float = 0.1):
        
        super().__init__()
        self.channels = channels
        self.height = height
        self.width = width
        
        # Branch A: Spatial path (VSS Blocks)
        self.vss_blocks = MambaBlockStack(
            channels, depth=depth, 
            expansion_ratio=expansion_ratio, 
            scan_dim=min(64, channels)
        )
        
        # Branch B: Spectral path (FFT-based filtering)
        self.spectral_gate = SpectralGating(
            channels, height, width, 
            threshold=threshold, 
            complex_init="kaiming"
        )
        
        # Fusion layer (learnable weighting)
        self.fusion_weight = nn.Parameter(torch.tensor(0.5))
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        # Branch A: Spatial context (VSS)
        spatial_out = self.vss_blocks(x)
        
        # Branch B: Frequency filtering (Spectral)
        spectral_out = self.spectral_gate(x)
        
        # Learnable fusion with sigmoid weight
        weight = torch.sigmoid(self.fusion_weight)
        output = weight * spatial_out + (1 - weight) * spectral_out
        
        return output

class PatchEmbedding(nn.Module):

    def __init__(self, in_channels: int = 3, out_channels: int = 64, 
                 patch_size: int = 4):
        
        super().__init__()
        self.patch_size = patch_size
        self.conv = nn.Conv2d(
            in_channels, out_channels,
            kernel_size=patch_size, stride=patch_size, bias=True
        )
        self.norm = nn.LayerNorm(out_channels)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        x = self.conv(x)
        B, C, H, W = x.shape
        x = x.permute(0, 2, 3, 1).reshape(B, H * W, C)
        x = self.norm(x)
        x = x.reshape(B, H, W, C).permute(0, 3, 3, 2)
        return x

class PatchMerging(nn.Module):

    def __init__(self, channels: int, out_channels: Optional[int] = None):
        
        super().__init__()
        out_channels = out_channels or channels * 2
        self.conv = nn.Conv2d(channels, out_channels, kernel_size=2, 
                              stride=2, bias=True)
        self.norm = nn.LayerNorm(out_channels)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        B, C, H, W = x.shape
        x = x.permute(0, 2, 3, 1).reshape(B, H * W, C)
        x = self.norm(x)
        x = x.reshape(B, H, W, C).permute(0, 3, 3, 2)
        return x

class PatchExpanding(nn.Module):

    def __init__(self, channels: int, out_channels: Optional[int] = None):
        
        super().__init__()
        out_channels = out_channels or channels // 2
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', 
                                    align_corners=True)
        self.conv = nn.Conv2d(channels, out_channels, kernel_size=1, bias=True)
        self.norm = nn.LayerNorm(out_channels)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.upsample(x)
        x = self.conv(x)
        B, C, H, W = x.shape
        x = x.permute(0, 2, 3, 1).reshape(B, H * W, C)
        x = self.norm(x)
        x = x.reshape(B, H, W, C).permute(0, 3, 3, 2)
        return x

class SpectralVMUNet(nn.Module):

    def __init__(self, in_channels: int = 1, out_channels: int = 3,
                 img_size: int = 256, base_channels: int = 64,
                 num_stages: int = 4, depth: int = 2):
        
        super().__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.img_size = img_size
        self.base_channels = base_channels
        self.num_stages = num_stages
        
        # Patch embedding
        self.patch_embed = PatchEmbedding(in_channels, base_channels, patch_size=4)
        initial_size = img_size // 4
        
        # Encoder
        self.encoder_blocks = nn.ModuleList()
        self.downsample_layers = nn.ModuleList()
        
        for i in range(num_stages):
            in_ch = base_channels * (2 ** i)
            out_ch = in_ch
            h = w = initial_size // (2 ** i)
            
            # SpectralVSSBlock
            block = SpectralVSSBlock(
                in_ch, h, w, depth=depth, expansion_ratio=2.0, threshold=0.1
            )
            self.encoder_blocks.append(block)
            
            # Downsampling (except after last encoder block)
            if i < num_stages - 1:
                down = PatchMerging(in_ch, in_ch * 2)
                self.downsample_layers.append(down)
        
        # Bottleneck - uses the last encoder's output channels
        # After num_stages-1 downsamplings, channels = base_channels * 2^(num_stages-1)
        bottleneck_ch = base_channels * (2 ** (num_stages - 1))
        bottleneck_h = bottleneck_w = initial_size // (2 ** (num_stages - 1))
        self.bottleneck = SpectralVSSBlock(
            bottleneck_ch, bottleneck_h, bottleneck_w,
            depth=depth + 1, expansion_ratio=2.0, threshold=0.1
        )
        
        # Decoder
        # We have num_stages - 1 decoder stages (matching skip connections)
        # Each decoder stage: upsample -> concat with skip -> fusion -> SpectralVSSBlock
        self.decoder_blocks = nn.ModuleList()
        self.upsample_layers = nn.ModuleList()
        
        num_decoder_stages = num_stages - 1
        
        for i in range(num_decoder_stages):
            # Going from deepest to shallowest
            # i=0: from bottleneck (8x8, 512ch) -> upsample to (16x16, 256ch)
            # i=1: from 16x16, 256ch -> upsample to (32x32, 128ch)
            # i=2: from 32x32, 128ch -> upsample to (64x64, 64ch)
            
            # Input channels: for i=0, it's bottleneck_ch; else from previous decoder output
            if i == 0:
                in_ch = bottleneck_ch  # 512 for default
            else:
                in_ch = base_channels * (2 ** (num_stages - 1 - i))
            
            # Output channels after upsampling
            out_ch = base_channels * (2 ** (num_stages - 2 - i))
            
            # Upsampling layer
            up = PatchExpanding(in_ch, out_ch)
            self.upsample_layers.append(up)
            
            # Skip connection comes from encoder at level (num_decoder_stages - 1 - i)
            # which has same spatial size after upsampling
            skip_ch = out_ch  # Skip has same channels as upsampled output
            
            # Spatial size at this level
            h = w = initial_size // (2 ** (num_stages - 2 - i))
            
            # Fusion: concatenate upsampled + skip, then reduce channels
            fused_ch = out_ch + skip_ch  # After concatenation
            fusion = nn.Sequential(
                nn.Conv2d(fused_ch, out_ch, kernel_size=1, bias=True),
                nn.GroupNorm(num_groups=min(32, out_ch), num_channels=out_ch, eps=1e-6)
            )
            self.decoder_blocks.append(fusion)
            
            # SpectralVSSBlock after fusion
            vss = SpectralVSSBlock(
                out_ch, h, w, depth=depth, expansion_ratio=2.0, threshold=0.1
            )
            self.decoder_blocks.append(vss)
        
        # Segmentation head
        self.seg_head = nn.Sequential(
            nn.Conv2d(base_channels, base_channels // 2, kernel_size=3, 
                      padding=1, bias=True),
            nn.GroupNorm(num_groups=32, num_channels=base_channels // 2, eps=1e-6),
            nn.GELU(),
            nn.Conv2d(base_channels // 2, out_channels, kernel_size=1, bias=True)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        # Patch embedding
        x = self.patch_embed(x)
        
        # Encoder path with skip connections storage
        # Skip connections are saved BEFORE downsampling
        skips = []
        for i in range(self.num_stages):
            x = self.encoder_blocks[i](x)
            # Save skip connection before downsampling
            if i < self.num_stages - 1:
                skips.append(x)
                x = self.downsample_layers[i](x)
        
        # The last encoder output goes to bottleneck (no skip for this level)
        # skips now contains: [stage0_out, stage1_out, stage2_out] for 4 stages
        
        # Bottleneck
        x = self.bottleneck(x)
        
        # Decoder path with skip connections
        # Decoder stages: num_stages - 1 (since last encoder has no skip)
        num_decoder_stages = self.num_stages - 1
        
        for i in range(num_decoder_stages):
            # Upsample
            x = self.upsample_layers[i](x)
            
            # Concatenate skip connection (in reverse order)
            # For i=0: skip from encoder stage num_stages-2 (last skip)
            # For i=1: skip from encoder stage num_stages-3
            skip_idx = num_decoder_stages - 1 - i
            skip = skips[skip_idx]
            x = torch.cat([x, skip], dim=1)
            
            # Fusion and processing
            x = self.decoder_blocks[2 * i](x)  # Fusion conv
            x = self.decoder_blocks[2 * i + 1](x)  # SpectralVSSBlock
        
        # Segmentation head
        output = self.seg_head(x)
        
        # Upsample to original resolution (since patch embedding uses stride 4)
        output = F.interpolate(output, size=(self.img_size, self.img_size),
                               mode='bilinear', align_corners=True)
        
        return output

if __name__ == "__main__":
    # Test the full architecture
    batch_size = 2
    in_channels = 1
    out_channels = 3  # Binary segmentation + background
    img_size = 256
    
    model = SpectralVMUNet(
        in_channels=in_channels,
        out_channels=out_channels,
        img_size=img_size,
        base_channels=64,
        num_stages=4,
        depth=2
    )
    
    # Create dummy input
    x = torch.randn(batch_size, in_channels, img_size, img_size)
    
    # Forward pass
    output = model(x)
    
    print(f"Input shape: {x.shape}")
    print(f"Output shape: {output.shape}")
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")



### 2.6 EGM-Net

### 2.7 Loss Functions

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional

class DiceLoss(nn.Module):

    def __init__(self, smooth: float = 1e-5, reduction: str = "mean"):
        
        super().__init__()
        self.smooth = smooth
        self.reduction = reduction
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        
        # Convert logits to probabilities
        pred = torch.softmax(pred, dim=1)
        
        # Ensure target has same shape as pred for multi-class
        if target.ndim == 3:  # (B, H, W) -> convert to one-hot
            target = F.one_hot(target.long(), num_classes=pred.shape[1])
            target = target.permute(0, 3, 3, 2).float()
        
        # Flatten spatial dimensions
        pred = pred.view(pred.shape[0], pred.shape[1], -1)
        target = target.view(target.shape[0], target.shape[1], -1)
        
        # Compute Dice score
        intersection = torch.sum(pred * target, dim=2)
        union = torch.sum(pred, dim=2) + torch.sum(target, dim=2)
        
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        
        # Return loss (1 - Dice)
        loss = 1.0 - dice
        
        if self.reduction == "mean":
            return loss.mean()
        elif self.reduction == "sum":
            return loss.sum()
        else:
            return loss

class FocalLoss(nn.Module):

    def __init__(self, alpha: float = 0.25, gamma: float = 2.0,
                 reduction: str = "mean"):
        
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        
        # Get class probabilities
        p = torch.softmax(pred, dim=1)
        
        # Get class log probabilities
        ce = F.cross_entropy(pred, target.long(), reduction='none')
        
        # Get probability of true class
        p_t = torch.gather(p, 3, target.long().unsqueeze(1)).squeeze(1)
        
        # Compute focal loss
        focal_weight = (1.0 - p_t) ** self.gamma
        focal_loss = focal_weight * ce
        
        if self.reduction == "mean":
            return focal_loss.mean()
        elif self.reduction == "sum":
            return focal_loss.sum()
        else:
            return focal_loss

class FrequencyLoss(nn.Module):

    def __init__(self, weight: float = 0.1):
        
        super().__init__()
        self.weight = weight
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        
        # Ensure both have batch and channel dimensions
        if pred.ndim == 3:
            pred = pred.unsqueeze(1)
        if target.ndim == 3:
            target = target.unsqueeze(1)
        
        # Flatten to single channel for FFT comparison
        if pred.shape[1] > 1:
            # For multi-channel, convert to grayscale by averaging
            pred = pred.mean(dim=1, keepdim=True)
        if target.shape[1] > 1:
            target = target.mean(dim=1, keepdim=True)
        
        # Apply FFT to convert to frequency domain
        pred_freq = torch.fft.rfft2(pred, dim=(-2, -1), norm="ortho")
        target_freq = torch.fft.rfft2(target, dim=(-2, -1), norm="ortho")
        
        # Compute L2 distance in frequency domain
        # Consider both magnitude and phase information
        loss_real = F.mse_loss(pred_freq.real, target_freq.real, reduction='mean')
        loss_imag = F.mse_loss(pred_freq.imag, target_freq.imag, reduction='mean')
        
        return loss_real + loss_imag

class SpectralDualLoss(nn.Module):

    def __init__(self, spatial_weight: float = 1.0, freq_weight: float = 0.1,
                 use_dice: bool = True, use_focal: bool = True):
        
        super().__init__()
        self.spatial_weight = spatial_weight
        self.freq_weight = freq_weight
        self.use_dice = use_dice
        self.use_focal = use_focal
        
        # Spatial losses
        if use_dice:
            self.dice_loss = DiceLoss(smooth=1e-5)
        
        if use_focal:
            self.focal_loss = FocalLoss(alpha=0.25, gamma=2.0)
        else:
            self.ce_loss = nn.CrossEntropyLoss()
        
        # Frequency loss
        self.freq_loss = FrequencyLoss(weight=freq_weight)
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor,
                return_components: bool = False) -> torch.Tensor:
        
        # Ensure target is on same device as pred
        target = target.to(pred.device)
        
        # Spatial losses
        spatial_loss = 0.0
        losses_dict = {}
        
        if self.use_dice:
            dice = self.dice_loss(pred, target)
            spatial_loss = spatial_loss + dice
            losses_dict['dice'] = dice.item()
        
        if self.use_focal:
            focal = self.focal_loss(pred, target)
            spatial_loss = spatial_loss + focal
            losses_dict['focal'] = focal.item()
        else:
            ce = self.ce_loss(pred, target)
            spatial_loss = spatial_loss + ce
            losses_dict['ce'] = ce.item()
        
        # Frequency loss
        # For frequency loss, we need to extract the predicted class (argmax) and compare
        pred_probs = torch.softmax(pred, dim=1)
        pred_class = torch.argmax(pred_probs, dim=1)  # (B, H, W)
        
        freq = self.freq_loss(pred_class.float(), target.float())
        losses_dict['freq'] = freq.item()
        
        # Weighted combination
        total_loss = (self.spatial_weight * spatial_loss + 
                     self.freq_weight * freq)
        losses_dict['total'] = total_loss.item()
        
        if return_components:
            return total_loss, losses_dict
        else:
            return total_loss

class BoundaryAwareLoss(nn.Module):

    def __init__(self, kernel_size: int = 3, weight: float = 1.0):
        
        super().__init__()
        self.kernel_size = kernel_size
        self.weight = weight
    
    def _compute_boundaries(self, mask: torch.Tensor) -> torch.Tensor:
        
        # Convert to float
        mask = mask.float().unsqueeze(1)  # (B, 3, H, W)
        
        # Compute gradients using Sobel-like operation
        kernel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                                dtype=mask.dtype, device=mask.device)
        kernel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]],
                                dtype=mask.dtype, device=mask.device)
        
        kernel_x = kernel_x.view(1, 3, 3, 3)
        kernel_y = kernel_y.view(1, 3, 3, 3)
        
        grad_x = F.conv2d(mask, kernel_x, padding=1)
        grad_y = F.conv2d(mask, kernel_y, padding=1)
        
        # Compute magnitude of gradient
        grad_magnitude = torch.sqrt(grad_x ** 2 + grad_y ** 2 + 1e-8)
        
        # Threshold to get boundary pixels
        boundary = (grad_magnitude > 0).float().squeeze(1)
        
        return boundary
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        
        # Get predicted class
        pred_probs = torch.softmax(pred, dim=1)
        pred_class = torch.argmax(pred_probs, dim=1)  # (B, H, W)
        
        # Compute boundary maps
        pred_boundary = self._compute_boundaries(pred_class)
        target_boundary = self._compute_boundaries(target)
        
        # Compute cross-entropy loss weighted by boundary
        ce_loss = F.cross_entropy(pred, target.long(), reduction='none')
        
        # Apply boundary weight (higher loss for boundary pixels)
        boundary_weight = (pred_boundary + target_boundary).clamp(0, 1)
        boundary_weight = 1.0 + boundary_weight  # Weight between 1 and 2
        
        weighted_loss = ce_loss * boundary_weight
        
        return weighted_loss.mean()

if __name__ == "__main__":
    # Test losses
    batch_size, num_classes, height, width = 2, 3, 64, 64
    
    # Create dummy predictions and targets
    pred = torch.randn(batch_size, num_classes, height, width)
    target = torch.randint(0, num_classes, (batch_size, height, width))
    
    # Test SpectralDualLoss
    loss_fn = SpectralDualLoss(spatial_weight=1.0, freq_weight=0.1)
    loss, components = loss_fn(pred, target, return_components=True)
    
    print(f"Total Loss: {loss.item():.4f}")
    for name, value in components.items():
        print(f"  {name}: {value:.4f}")
    
    # Test BoundaryAwareLoss
    boundary_loss_fn = BoundaryAwareLoss()
    boundary_loss = boundary_loss_fn(pred, target)
    print(f"\nBoundary Loss: {boundary_loss.item():.4f}")



### 2.8 Metrics

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from monai.metrics import compute_hausdorff_distance

def count_parameters(model):
    
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class SegmentationMetrics:
    def __init__(self, num_classes, device):
        self.num_classes = num_classes
        self.device = device
        self.reset()
        
    def reset(self):
        self.batches = 0
        self.total_correct_pixels = 0
        self.total_pixels = 0
        
        # Aggregated stats for Precision/Recall/F1 (Global)
        self.tp = torch.zeros(self.num_classes, device=self.device)
        self.fp = torch.zeros(self.num_classes, device=self.device)
        self.fn = torch.zeros(self.num_classes, device=self.device)
        
        # Accumulators for averaging Batch-wise metrics
        self.dice_sum = torch.zeros(self.num_classes, device=self.device)
        self.iou_sum = torch.zeros(self.num_classes, device=self.device)
        self.hd95_sum = torch.zeros(self.num_classes, device=self.device)
        
        # Track valid batches for HD95 (it can be NaN if class is missing)
        self.hd95_counts = torch.zeros(self.num_classes, device=self.device)

    def update(self, preds, targets):
        
        self.batches += 1
        
        # Accuracy
        self.total_correct_pixels += (preds == targets).sum().item()
        self.total_pixels += targets.numel()
        
        # Create one-hot for HD95 and Dice
        # preds_oh: (B, C, H, W)
        preds_oh = F.one_hot(preds, num_classes=self.num_classes).permute(0, 3, 1, 2).float()
        targets_oh = F.one_hot(targets, num_classes=self.num_classes).permute(0, 3, 1, 2).float()
        
        # Helper for Dice/IoU/TP/FP/FN
        for c in range(self.num_classes):
            p_flat = preds_oh[:, c].reshape(-1)
            t_flat = targets_oh[:, c].reshape(-1)
            
            intersection = (p_flat * t_flat).sum()
            union = p_flat.sum() + t_flat.sum()
            
            # Global TP/FP/FN accumulation
            self.tp[c] += intersection
            self.fp[c] += (p_flat.sum() - intersection)
            self.fn[c] += (t_flat.sum() - intersection)
            
            # Batch-wise Dice/IoU accumulation
            dice = (2. * intersection + 1e-6) / (union + 1e-6)
            iou = (intersection + 1e-6) / (union - intersection + 1e-6)
            
            self.dice_sum[c] += dice
            self.iou_sum[c] += iou
            
        # HD95 Compliance (MONAI)
        # compute_hausdorff_distance expects (B, C, spatial...)
        # include_background=True usually, but we iterate.
        # We can compute all classes at once.
        try:
            import warnings
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                
                # percentile=95
                hd95_batch = compute_hausdorff_distance(
                    y_pred=preds_oh, 
                    y=targets_oh, 
                    include_background=True, 
                    percentile=95.0,
                    spacing=None  # Pixel space
                )
            # hd95_batch is (B, C)
            
            for c in range(self.num_classes):
                # Filter NaNs/Infs (happens if class missing in both pred and target, or just one)
                # MONAI returns NaN if one is empty. We mostly care if target exists.
                # Common practice: if target is empty, skip. If target exists but pred empty, HD is high (inf).
                # MONAI behavior: Nan if both empty. Inf if one empty.
                
                valid_vals = hd95_batch[:, c]
                valid_mask = ~torch.isnan(valid_vals) & ~torch.isinf(valid_vals)
                
                if valid_mask.any():
                    self.hd95_sum[c] += valid_vals[valid_mask].sum()
                    self.hd95_counts[c] += valid_mask.sum()
                    
        except Exception as e:
            # Fallback or strict error? 
            # Often happens if shapes are weird or empty batch.
            pass

    def compute(self):
        
        metrics = {}
        
        # Global Accuracy
        metrics['accuracy'] = self.total_correct_pixels / max(self.total_pixels, 1)
        
        # Per-class metrics
        dice_scores = []
        iou_scores = []
        hd95_scores = []
        precision_scores = []
        recall_scores = []
        f1_scores = []
        
        for c in range(self.num_classes):
            # Batch-averaged Dice/IoU
            dice_scores.append((self.dice_sum[c] / max(self.batches, 1)).item())
            iou_scores.append((self.iou_sum[c] / max(self.batches, 1)).item())
            
            # Batch-averaged HD95
            if self.hd95_counts[c] > 0:
                hd95_scores.append((self.hd95_sum[c] / self.hd95_counts[c]).item())
            else:
                hd95_scores.append(float('nan')) # Or 0.0 or inf
            
            # Global-based Precision/Recall/F1
            p = (self.tp[c] / (self.tp[c] + self.fp[c] + 1e-6)).item()
            r = (self.tp[c] / (self.tp[c] + self.fn[c] + 1e-6)).item()
            f1 = 2 * p * r / (p + r + 1e-6) if (p + r) > 0 else 0.0
            
            precision_scores.append(p)
            recall_scores.append(r)
            f1_scores.append(f1)
            
        metrics['dice_scores'] = dice_scores
        metrics['iou'] = iou_scores
        metrics['hd95'] = hd95_scores
        metrics['precision'] = precision_scores
        metrics['recall'] = recall_scores
        metrics['f1_score'] = f1_scores
        
        return metrics


In [ ]:
print('✅ All modules loaded!')

---
## 3️⃣ Dataset

Download and preprocess ACDC cardiac MRI dataset.

### 3.1 Configuration

In [ ]:
DATASET = 'ACDC'
DRIVE_FOLDER_ID = '1EelzBVjIoDQ4uzt0_2JzmF_PuUHsD93e'
RAW_DATA_DIR = f'./data/{DATASET}'
PREPROCESSED_DIR = f'./preprocessed_data/{DATASET}'
IMG_SIZE = 224
NUM_CLASSES = 4
CLASS_NAMES = ['Background', 'RV', 'Myocardium', 'LV']

os.makedirs(RAW_DATA_DIR, exist_ok=True)
print(f"📊 Dataset: {DATASET}")
print(f"   Classes: {CLASS_NAMES}")

### 3.2 Download from Kaggle

In [ ]:
import kagglehub
acdc_path = kagglehub.dataset_download('samdazel/automated-cardiac-diagnosis-challenge-miccai17')

print('Data source import complete.')
print(f'Data path: {acdc_path}')
# Xem cấu trúc thư mục
import os
for item in os.listdir(acdc_path):
    print(f'  {item}')


RAW_DATA_DIR = os.path.join(acdc_path, 'database')
print(f'RAW_DATA_DIR: {RAW_DATA_DIR}')
# Xem cấu trúc
for item in os.listdir(RAW_DATA_DIR):
    print(f'  {item}')

### 3.3 Preprocess (NIfTI → NumPy)

In [ ]:
import os
import sys
import configparser
import numpy as np
import nibabel as nib
from tqdm import tqdm
import json
from skimage.transform import resize

# Add project root to path

def normalize_intensity(image):
    
    # Clip outliers
    p05 = np.percentile(image, 0.5)
    p995 = np.percentile(image, 99.5)
    image = np.clip(image, p05, p995)
    
    # Z-score
    mean = np.mean(image)
    std = np.std(image)
    if std > 0:
        return (image - mean) / std
    return image

def preprocess_single_patient_acdc(patient_path, output_dir, target_size=(224, 224)):
    
    patient_folder = os.path.basename(patient_path)
    info_cfg_path = os.path.join(patient_path, 'Info.cfg')
    
    # Tạo folder output cho slice
    img_save_dir = os.path.join(output_dir, 'images')
    mask_save_dir = os.path.join(output_dir, 'masks')
    os.makedirs(img_save_dir, exist_ok=True)
    os.makedirs(mask_save_dir, exist_ok=True)
    
    # Đọc config để biết frame nào là ED, frame nào là ES
    if not os.path.exists(info_cfg_path):
        return 0
    
    try:
        parser = configparser.ConfigParser()
        with open(info_cfg_path, 'r') as f:
            config_string = '[DEFAULT]\n' + f.read()
        parser.read_string(config_string)
        ed_frame = int(parser['DEFAULT']['ED'])
        es_frame = int(parser['DEFAULT']['ES'])
    except Exception as e:
        print(f"  Error reading Info.cfg for {patient_folder}: {e}")
        return 0
    
    slices_saved = 0
    
    for frame_num, frame_name in [(ed_frame, 'ED'), (es_frame, 'ES')]:
        img_filename = f'{patient_folder}_frame{frame_num:02d}.nii.gz'
        mask_filename = f'{patient_folder}_frame{frame_num:02d}_gt.nii.gz'
        
        # Tìm file (support cả .nii và .nii.gz)
        img_path = None
        mask_path = None
        for suffix in ['.gz', '']:
            test_img = os.path.join(patient_path, img_filename.replace('.gz', '') if suffix == '' else img_filename)
            test_mask = os.path.join(patient_path, mask_filename.replace('.gz', '') if suffix == '' else mask_filename)
            if os.path.exists(test_img):
                img_path = test_img
                mask_path = test_mask
                break
        
        if img_path is None or not os.path.exists(img_path):
            continue
            
        try:
            # Load NIfTI
            img_nii = nib.load(img_path)
            img_data = img_nii.get_fdata() # (H, W, D)
            
            mask_data = None
            if os.path.exists(mask_path):
                mask_data = nib.load(mask_path).get_fdata()
            else:
                continue # Bỏ qua nếu không có mask
            
            # 1. Normalize Intensity TRƯỚC khi resize (tính trên toàn volume 3D)
            img_data = normalize_intensity(img_data)
            
            num_slices = img_data.shape[2]
            
            # 2. Xử lý từng slice và lưu ngay lập tức
            for i in range(num_slices):
                slice_img = img_data[:, :, i]
                slice_mask = mask_data[:, :, i]
                
                # Bỏ qua slice đen thui (không có thông tin) để tránh nhiễu training
                if np.sum(slice_img) == 0:
                    continue
                
                # Resize (Lưu ý: resize của skimage range input=output, đã normalize thì vẫn giữ range)
                slice_img_resized = resize(
                    slice_img, target_size, order=1, preserve_range=True, anti_aliasing=True, mode='reflect'
                ).astype(np.float32)
                
                slice_mask_resized = resize(
                    slice_mask, target_size, order=0, preserve_range=True, anti_aliasing=False, mode='reflect'
                ).astype(np.uint8) # Mask phải là int
                
                # Tạo tên file: patient001_ED_slice005.npy
                file_id = f"{patient_folder}_{frame_name}_slice{i:03d}"
                
                np.save(os.path.join(img_save_dir, f"{file_id}.npy"), slice_img_resized)
                np.save(os.path.join(mask_save_dir, f"{file_id}.npy"), slice_mask_resized)
                
                slices_saved += 1
                
        except Exception as e:
            print(f"  Error processing {patient_folder} frame {frame_num}: {e}")
            continue
            
    return slices_saved

def preprocess_acdc_dataset(input_dir, output_dir, target_size=(224, 224)):
    
    # Lấy danh sách patient
    patient_folders = sorted([
        os.path.join(input_dir, d) 
        for d in os.listdir(input_dir) 
        if os.path.isdir(os.path.join(input_dir, d)) and d.startswith('patient')
    ])
    
    print(f"Found {len(patient_folders)} patients. Outputting 2D slices to {output_dir}")
    
    total_slices = 0
    
    for patient_path in tqdm(patient_folders, desc="Processing ACDC"):
        slices = preprocess_single_patient_acdc(patient_path, output_dir, target_size)
        total_slices += slices
        
    print(f"\nCompleted! Saved {total_slices} slices total.")
    print(f"Images: {os.path.join(output_dir, 'images')}")
    print(f"Masks:  {os.path.join(output_dir, 'masks')}")


training_dir = os.path.join(acdc_path, 'database', 'training')
output_dir = './preprocessed_data/ACDC'
print(f"📂 Input: {training_dir}")
print(f"📂 Output: {output_dir}")
preprocess_acdc_dataset(training_dir, output_dir, target_size=(224, 224))

### 3.4 Create DataLoaders

In [ ]:
from torch.utils.data import Dataset, DataLoader
import glob

class ACDCDataset2D(Dataset):
    """Fast memmap-based dataset for preprocessed 2D slices."""
    
    def __init__(self, data_dir, split='train'):
        img_dir = os.path.join(data_dir, 'images')
        all_files = sorted(glob.glob(os.path.join(img_dir, '*.npy')))
        
        np.random.seed(42)
        indices = np.random.permutation(len(all_files))
        split_idx = int(0.8 * len(all_files))
        
        if split == 'train':
            self.files = [all_files[i] for i in indices[:split_idx]]
        else:
            self.files = [all_files[i] for i in indices[split_idx:]]
        
        self.mask_dir = os.path.join(data_dir, 'masks')
        print(f"   {split.upper()}: {len(self.files)} 2D slices")
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_path = self.files[idx]
        mask_path = os.path.join(self.mask_dir, os.path.basename(img_path))
        
        # memmap for fast disk access
        img = np.load(img_path, mmap_mode='r').copy()
        seg = np.load(mask_path, mmap_mode='r').copy()
        
        # Already 2D (H, W) -> add channel dim -> (1, H, W)
        return torch.from_numpy(img).unsqueeze(0).float(), torch.from_numpy(seg).long()

BATCH_SIZE = 4
train_ds = ACDCDataset2D(PREPROCESSED_DIR, 'train')
val_ds = ACDCDataset2D(PREPROCESSED_DIR, 'val')
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
print(f"✅ Memmap DataLoader ready")

---
## 4️⃣ Training

### 4.1 Create Model

In [ ]:
model = EGMNet(use_hrnet=True, in_channels=3, num_classes=NUM_CLASSES, img_size=IMG_SIZE).to(device)
print(f"✅ EGM-Net: {sum(p.numel() for p in model.parameters()):,} parameters")


### 4.2 Loss & Optimizer

In [ ]:
criterion = SpectralDualLoss(spatial_weight=1.0, freq_weight=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

### 4.3 Training Loop

In [ ]:
# === TRAINING ===
from torch.cuda.amp import autocast, GradScaler

NUM_EPOCHS = 100
LEARNING_RATE = 1e-4
EARLY_STOP_PATIENCE = 20
NUM_CLASSES = 4
CLASS_NAMES = {0: 'BG', 1: 'RV', 2: 'MYO', 3: 'LV'}

print(f"Device: {device}")
model = EGMNet(use_hrnet=True, in_channels=3, num_classes=NUM_CLASSES, img_size=IMG_SIZE).to(device)
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

criterion = SpectralDualLoss(spatial_weight=1.0, freq_weight=0.1).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10)
scaler = GradScaler()

best_dice = 0.0
epochs_no_improve = 0

for epoch in range(NUM_EPOCHS):
    torch.cuda.empty_cache()
    
    model.train()
    train_loss = 0.0
    for images, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        
        with autocast():
            outputs = model(images)
            pred = outputs['output'] if isinstance(outputs, dict) else outputs
            loss = criterion(pred, masks)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
    
    torch.cuda.empty_cache()
    
    model.eval()
    val_dice_sum = 0
    val_count = 0
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            with autocast():
                outputs = model(images)
            pred = outputs['output'] if isinstance(outputs, dict) else outputs
            pred_cls = pred.argmax(dim=1)
            for c in range(1, NUM_CLASSES):
                inter = ((pred_cls == c) & (masks == c)).sum()
                union = (pred_cls == c).sum() + (masks == c).sum()
                val_dice_sum += (2 * inter / (union + 1e-5)).item()
            val_count += 1
    
    avg_dice = val_dice_sum / (val_count * (NUM_CLASSES - 1))
    scheduler.step(avg_dice)
    
    print(f"Epoch {epoch+1}: Loss={train_loss/len(train_loader):.4f}, Dice={avg_dice:.4f}")
    
    if avg_dice > best_dice:
        best_dice = avg_dice
        torch.save(model.state_dict(), 'best_model.pth')
        print(f"   ✅ Best model saved!")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
    
    if epochs_no_improve >= EARLY_STOP_PATIENCE:
        print(f"Early stopping!")
        break

print(f"Training complete! Best Dice: {best_dice:.4f}")


: 

In [ ]:
# === EVALUATE FUNCTION ===
def evaluate_metrics(model, dataloader, device, num_classes):
    """Evaluate model on a dataloader and return metrics."""
    model.eval()
    metrics_tracker = SegmentationMetrics(num_classes=num_classes, device=device)
    
    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            pred = outputs['output'] if isinstance(outputs, dict) else outputs
            pred_cls = pred.argmax(dim=1)
            metrics_tracker.update(pred_cls, masks)
    
    return metrics_tracker.compute()

# === FINAL TEST EVALUATION ===
def final_evaluation(model, test_loader, device, num_classes, class_names):
    """Run final evaluation on test set."""
    print("\n" + "="*60)
    print("📊 FINAL TEST EVALUATION")
    print("="*60)
    
    # Load best model
    checkpoint = torch.load('best_model.pth')
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded best model from epoch {checkpoint['epoch']}")
    
    metrics = evaluate_metrics(model, test_loader, device, num_classes)
    
    print("\n--- Per-Class Results ---")
    for c in range(num_classes):
        print(f"   {class_names[c]:<5}: Dice={metrics['dice_scores'][c]:.4f}, "
              f"IoU={metrics['iou'][c]:.4f}, HD95={metrics['hd95'][c]:.2f}")
    
    avg_dice = np.mean(metrics['dice_scores'][1:])
    avg_iou = np.mean(metrics['iou'][1:])
    avg_hd95 = np.nanmean(metrics['hd95'][1:])
    
    print("\n--- Summary (Foreground Only) ---")
    print(f"   Avg Dice: {avg_dice:.4f}")
    print(f"   Avg IoU:  {avg_iou:.4f}")
    print(f"   Avg HD95: {avg_hd95:.2f}")
    print(f"   Accuracy: {metrics['accuracy']:.4f}")
    
    return metrics

In [ ]:
# Final evaluation on test set
final_metrics = final_evaluation(model, val_loader, device, NUM_CLASSES, CLASS_NAMES)

---
## 5️⃣ Visualization

In [ ]:
model.eval()
images, masks = next(iter(val_loader))
images, masks = images.to(device), masks.to(device)

with torch.no_grad():
    outputs = model(images)
    pred = outputs['output'] if isinstance(outputs, dict) else outputs
    pred_cls = pred.argmax(dim=1)

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
for i in range(4):
    axes[0, i].imshow(images[i, 0].cpu(), cmap='gray')
    axes[0, i].set_title('Input')
    axes[1, i].imshow(masks[i].cpu(), cmap='jet', vmin=0, vmax=NUM_CLASSES-1)
    axes[1, i].set_title('Ground Truth')
    axes[2, i].imshow(pred_cls[i].cpu(), cmap='jet', vmin=0, vmax=NUM_CLASSES-1)
    axes[2, i].set_title('Prediction')
for ax in axes.flatten():
    ax.axis('off')
plt.tight_layout()
plt.show()